In [1]:
import numpy as np
import nibabel as nib
import os,sys
import math
import os.path
import random
import pickle
import time
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["SM_FRAMEWORK"] = "tf.keras"
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import MaxPooling2D,MaxPooling3D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K
from tensorflow.keras.layers import GaussianNoise


KeyboardInterrupt: 

In [ ]:
import skimage
from tensorflow.keras.models import load_model
from skimage.transform import resize
from skimage import io, transform

In [ ]:
d3_plat_1 = [[0,0,0],[-1,-1,0],[0,-1,0],[1,-1,0],[1,0,0],[1,1,0],[0,1,0],[-1,1,0],[-1,0,0],
             [0,0,1],[-1,-1,1],[0,-1,1],[1,-1,1],[1,0,1],[1,1,1],[0,1,1],[-1,1,1],[-1,0,1],
             [0,0,-1],[-1,-1,-1],[0,-1,-1],[1,-1,-1],[1,0,-1],[1,1,-1],[0,1,-1],[-1,1,-1],[-1,0,-1]]
d2_plat_1 = [[0,0,0],[-1,-1,0],[0,-1,0],[1,-1,0],[1,0,0],[1,1,0],[0,1,0],[-1,1,0],[-1,0,0]]
d3_plat_1_5 = [[0,0,0],[-5,-5,0],[0,-5,0],[5,-5,0],[5,0,0],[5,5,0],[0,5,0],[-5,5,0],[-5,0,0],
               [0,0,5],[-5,-5,5],[0,-5,5],[5,-5,5],[5,0,5],[5,5,5],[0,5,5],[-5,5,5],[-5,0,5],
               [0,0,-5],[-5,-5,-5],[0,-5,-5],[5,-5,-5],[5,0,-5],[5,5,-5],[0,5,-5],[-5,5,-5],[-5,0,-5]]

patch0 = 25
patch1 = 25
patch2 = 25
pdt_patch0 = 5
pdt_patch1 = 5
pdt_patch2 = 5

In [ ]:
def read_data(path):#定义一个“读取图像”函数，用来读取图像
    image_data = nib.load(path)
    return image_data

In [ ]:
path_1 = #图像路径，日后可以根据需要修改
target_1 = read_data(path_1)#使用“读取图像”函数

target_3d_1 = target_1.get_fdata()
target_3d_1.shape

In [ ]:
origin0 = 0.23
origin1 = 0.23
origin2 = 0.36

insert0 = origin0/0.1
insert1 = origin1/0.1
insert2 = origin2/0.1

def bilinear(org_img, org_shape, dst_shape):
    dst_img = np.zeros((dst_shape[0], dst_shape[1], dst_shape[2]))
    dst_h, dst_w, dst_d = dst_shape
    org_h, org_w, org_d = org_shape
    for i in range(dst_h):
        print(i)
        for j in range(dst_w):
            for k in range(dst_d):
                src_x = j * float(org_w / dst_w)
                src_y = i * float(org_h / dst_h)
                src_z = k * float(org_d / dst_d)
                src_x_int = j * org_w // dst_w
                src_y_int = i * org_h // dst_h
                src_z_int = k * org_d // dst_d
                a = src_x - src_x_int
                b = src_y - src_y_int
                c = src_z - src_z_int

                if src_x_int+1 == org_w or src_y_int+1 == org_h or src_z_int+1 == org_d:
                    dst_img[i, j, k] = org_img[src_y_int, src_x_int, src_z_int]
                    continue
                # print(src_x_int, src_y_int)
                dst_img[i, j, k] = (1. - a) * (1. - b) * (1. - c) * org_img[src_y_int+1, src_x_int+1, src_z_int] + \
                                   (1. - a) * b * (1. - c) * org_img[src_y_int, src_x_int+1, src_z_int] + \
                                    a * (1. - b) * (1. - c) * org_img[src_y_int+1, src_x_int, src_z_int] + \
                                    a * b * (1. - c) * org_img[src_y_int, src_x_int, src_z_int] + \
                                    (1. - a) * (1. - b) * c * org_img[src_y_int+1, src_x_int+1, src_z_int] + \
                                   (1. - a) * b * c * org_img[src_y_int, src_x_int+1, src_z_int] + \
                                    a * (1. - b) * c * org_img[src_y_int+1, src_x_int, src_z_int] + \
                                    a * b * c * org_img[src_y_int, src_x_int, src_z_int]
    
    return dst_img

In [ ]:
def Bicubic_Bell( num ):
   # print( num)
    if  -1.5 <= num <= -0.5:
      #  print( -0.5 * ( num + 1.5) ** 2 )
        return -0.5 * ( num + 1.5) ** 2
    if -0.5 < num <= 0.5:
       # print( 3/4 - num ** 2 )
        return 3/4 - num ** 2
    if 0.5 < num <= 1.5:
       # print( 0.5 * ( num - 1.5 ) ** 2 )
        return 0.5 * ( num - 1.5 ) ** 2
    else:
       # print( 0 )
        return 0
        
    
def Bicubic (img):
    height,width,channel = img.shape
    bigger_height = int(height*insert0+1)
    bigger_width = int(width*insert1+1)
    bigger_channel = int(channel*insert2+1)
    Bicubic_img = np.zeros((bigger_height,bigger_width,bigger_channel), dtype=np.uint8)
    
    for i in range(0,bigger_height):
        print(i)
        for j in range(0,bigger_width):
            for k in range(0,bigger_channel):
                row = ( i / bigger_height ) * img.shape[0]
                col = ( j / bigger_width ) * img.shape[1]
                slc = (k/bigger_channel)*img.shape[2]
                row_int = int( row )
                col_int = int( col )
                slc_int = int(slc)
                u = row - row_int
                v = col - col_int
                w = slc-slc_int
                tmp = 0
                for m in range( -1, 3):
                    for n in range( -1, 3 ):
                        for o in range(-1,3):
                            if ( row_int + m ) < 0 or (col_int+n) < 0 or ( slc_int + o ) < 0 or ( row_int + m ) >= img.shape[0] or (col_int+n) >= img.shape[1] or (slc_int+o) >= img.shape[2]:
                                row_int = img.shape[0] - 1 - m
                                col_int = img.shape[1] - 1 - n
                                slc_int = img.shape[2] - 1 - o

                            numm = img[row_int + m][col_int+n][slc_int+o] * Bicubic_Bell( m-u ) * Bicubic_Bell( n-v ) * Bicubic_Bell( o-w )
                            tmp += np.abs( np.trunc( numm ) )

                Bicubic_img[i][j][k] = tmp
    return Bicubic_img

In [ ]:
target_3d_normalization=((target_3d_1/1000))*255
#other images

In [ ]:
#S57TRP
positive_train = [
    [380, 497, 14], [380, 496, 14], [379, 498, 14], [379, 497, 14], [379, 496, 14], [379, 495, 14], [378, 499, 14], [378, 498, 14], 
    [378, 497, 14], [377, 499, 14], [377, 498, 14], [377, 498, 14], [377, 497, 14], [376, 500, 14], [376, 499, 14], [376, 498, 14], 
    [375, 500, 13], [375, 499, 13], [375, 498, 13], [375, 501, 13], [333, 451, 26], [332, 451, 26], [338, 450, 26], [337, 450, 26], 
    [332, 450, 26], [338, 449, 26], [337, 449, 26], [336, 449, 26], [338, 448, 26], [337, 448, 26], [338, 447, 26], [337, 447, 26], 
    [333, 447, 26], [337, 446, 26], [333, 446, 26], [332, 446, 26], [331, 446, 26], [332, 444, 26], [332, 443, 26], [331, 443, 26],
    [330, 432, 26],[329, 432, 26],[328, 432, 26],[330, 431, 26],[329, 431, 26],[328, 431, 26],[330, 430, 26],[329, 430, 26],[328, 430, 26],[329, 429, 26],
    [339, 464, 27],[338, 463, 27],[339, 463, 27],[338, 462, 27],[339, 462, 27],[337, 462, 27],[337, 461, 27],[337, 460, 27],[336, 460, 27],[335, 458, 27],
    [335, 457, 27],[335, 456, 27],[334, 456, 27],[334, 455, 27],[334, 454, 27],[336, 449, 27],[335, 448, 27],[334, 448, 27],[333, 448, 27],[338, 446, 27],
    [337, 446, 27],[332, 446, 27],[337, 445, 27],[328, 444, 27],[327, 443, 27],[327, 442, 27],[327, 441, 27],[327, 440, 27],[328, 439, 27],[330, 438, 27],
    [330, 437, 27],[331, 437, 27],[332, 437, 27],[333, 437, 27],[334, 437, 27],[330, 436, 27],[329, 435, 27],[329, 434, 27],[330, 433, 27],[329, 433, 27],
    [330, 432, 27],[329, 432, 27],[330, 431, 27],[329, 431, 27],[330, 430, 27],[329, 430, 27],[330, 429, 27],[329, 429, 27],[331, 428, 27],[330, 428, 27],
    [329, 428, 27],[330, 427, 27],[329, 427, 27],[330, 426, 27],[330, 425, 27],[334, 448, 28],[333, 448, 28],[333, 447, 28],[332, 447, 28],[338, 446, 28],
    [332, 446, 28],[333, 446, 28],[338, 445, 28],[332, 445, 28],[334, 437, 28],[330, 430, 28],[330, 429, 28],[330, 428, 28],[329, 428, 28],[331, 427, 28],
    [331, 442, 26], [331, 441, 26], [330, 441, 26], [331, 440, 26], [330, 440, 26], [331, 439, 26], [330, 439, 26], [331, 438, 26], 
        [331, 425, 29],[332, 424, 29],[331, 424, 29],[333, 423, 29],[332, 423, 29],[331, 423, 29],[334, 422, 29],[333, 422, 29],[332, 422, 29],[331, 422, 29],
    [334, 421, 29],[333, 421, 29],[332, 421, 29],[375, 473, 29],[375, 472, 29],[374, 472, 29],[365, 467, 29],[364, 467, 29],[365, 466, 29],[364, 466, 29],
    [364, 465, 30],[364, 465, 31],[364, 458, 28],[365, 458, 28],[364, 458, 29],[368, 458, 29],[368, 457, 29],[364, 457, 29],[368, 457, 30],[364, 458, 30],
    [330, 438, 26], [329, 438, 26], [330, 437, 26], [329, 437, 26], [330, 436, 26], [329, 436, 26], [330, 435, 26], [329, 435, 26], 
    [330, 434, 26], [329, 434, 26], [330, 433, 26], [329, 433, 26], [444, 523, 36], [445, 523, 36], [446, 523, 36], [448, 524, 36], 
    [449, 524, 36], [450, 524, 36], [451, 525, 36], [452, 526, 36], [453, 526, 36], [454, 527, 36], [455, 529, 37], [456, 531, 38], 
    [456, 532, 39], [456, 533, 39], [457, 534, 40], [461, 543, 43], [457, 535, 40], [458, 536, 41], [458, 537, 41], [459, 538, 42], 
    [459, 539, 42], [460, 540, 42], [460, 541, 42], [460, 542, 43], [460, 543, 43], [461, 544, 44], [461, 545, 44], [461, 546, 44], 
    [460, 547, 45], [460, 548, 45], [460, 549, 46], [437, 524, 37], [436, 524, 38], [435, 524, 39], [434, 523, 39], [433, 523, 40], 
    [432, 523, 41], [431, 523, 42], [383, 496, 14], [383, 495, 14], [382, 496, 14], [382, 495, 14], [382, 497, 14], [381, 497, 14], 
    [381, 496, 14], [381, 495, 14], [340, 465, 20], [339, 465, 20], [338, 465, 20], [340, 464, 20], [339, 464, 20], [339, 463, 20], 
    [339, 462, 20], [338, 462, 20], [339, 461, 20], [338, 461, 20], [337, 461, 20], [339, 460, 20], [338, 460, 20], [339, 459, 20], 
    [338, 459, 20], [337, 459, 20], [339, 458, 20], [338, 458, 20], [337, 458, 20], [338, 457, 21], [337, 457, 21], [336, 457, 21], 
    [338, 456, 21], [337, 456, 21], [336, 456, 21], [338, 455, 21], [337, 455, 21], [336, 455, 21], [338, 454, 22], [337, 454, 22], 
    [336, 454, 22], [338, 453, 22], [337, 453, 22], [336, 453, 22], [337, 452, 23], [336, 452, 23], [335, 452, 23], [337, 451, 23], 
    [336, 451, 23], [335, 451, 23], [339, 452, 24], [338, 452, 24], [336, 452, 24], [335, 452, 24], [334, 452, 24], [339, 451, 24], 
    [338, 451, 24], [336, 451, 24], [335, 451, 24], [334, 451, 24], [336, 450, 24], [335, 450, 24], [334, 450, 24], [335, 449, 24], 
    [334, 449, 24], [336, 449, 24], [333, 449, 24], [335, 448, 24], [334, 448, 24], [333, 448, 24], [335, 447, 24], [334, 447, 24], 
    [333, 447, 24], [334, 446, 24], [333, 446, 24], [332, 446, 24], [334, 445, 24], [333, 445, 24], [332, 445, 24], [339, 452, 25], 
    [338, 452, 25], [339, 451, 25], [338, 451, 25], [339, 450, 25], [338, 450, 25], [337, 450, 25], [338, 449, 25], [337, 449, 25], 
    [334, 449, 25], [339, 448, 25], [338, 448, 25], [337, 448, 25], [334, 448, 25], [333, 448, 25], [334, 447, 25], [333, 447, 25], 
    [332, 447, 25], [331, 447, 25], [334, 446, 25], [333, 446, 25], [332, 446, 25], [331, 446, 25], [333, 445, 25], [332, 445, 25], 
    [331, 445, 25], [330, 445, 25], [333, 444, 25], [332, 444, 25], [331, 444, 25], [330, 444, 25], [332, 443, 25]
]

In [ ]:
#S67TRP
positive_train67 =  [
    [261,387,23],[265, 385, 24],[265, 383, 24],[237,387,21],[269, 386, 23],[268, 386, 23],[264, 386, 23],[263, 386, 23],[262, 386, 23],
    [261, 386, 23],[260, 386, 23],[259, 386, 23],[258, 386, 23],[257, 386, 23],[256, 386, 23],[255, 386, 23],[254, 386, 23],[253, 386, 23],
    [252, 386, 23],[251, 386, 23],[296, 382, 29],[296, 381, 29],[295,384,28],[293, 382, 29],[309, 383, 29],[308, 383, 29],[307, 383, 29],
    [306, 383, 29],[305, 383, 29],[305, 383, 29],[304, 383, 29],[303, 383, 29],[302, 383, 29],[301, 383, 29],[300, 383, 29],[299, 383, 29],
    [298, 383, 29],[297, 383, 29],[296, 383, 29],[295, 383, 29],[294, 383, 29],[293, 383, 29],[292, 383, 29],[291, 383, 29],[290, 383, 29],
    [289, 383, 29],[288, 383, 29],[287, 383, 29],[291, 386, 29],[291, 385, 29],[291, 384, 29],[291, 383, 29],[291, 382, 29],[291, 381, 29],
    [291, 383, 30],[291, 383, 28],[65, 496, 83],[63, 496, 83],[64, 495, 86],[64, 494, 86],[64, 493, 86],[64, 492, 86],[64, 491, 86],
    [65, 496, 86],[65, 495, 86],[65, 494, 86],[65, 493, 86],[65, 492, 86],[65, 491, 86],[65, 490, 86],[367,461,32],[366, 461, 32],
    [367, 460, 32],[364, 461, 32],[367, 462, 31],[367, 461, 31],[367, 460, 31],[367, 459, 31],[367, 461, 30],[366, 461, 30],[365, 461, 30],
    [364, 461, 30],[363, 461, 30],[277, 492, 56],[277, 491, 56],[277, 491, 54],[277, 490, 54],[277, 489, 54],[277, 488, 54],[277, 495, 56],
    [277, 495, 55],[277, 495, 54],[277, 495, 53],[277, 495, 52],[277, 495, 51],[278, 493, 59],[278, 493, 58],[278, 493, 57],[278, 493, 56],
    [278, 493, 55],[278, 493, 54],[278, 493, 53],[278, 493, 52],[278, 493, 51],[283, 540, 110],[283, 538, 110],[283, 538, 110],
    [283, 533, 110],[283, 541, 109],[283, 540, 109],[283, 539, 109],[283, 538, 109],[283, 537, 109],[283, 536, 109],[282, 543, 109],
    [282, 542, 109],[282, 541, 109],[282, 540, 109],[282, 539, 109],[282, 538, 109],[282, 537, 109],[282, 536, 109],[282, 535, 109],
    [282, 534, 109],[282, 533, 109],[282, 539, 109],[281, 539, 109],[280, 539, 109],[279, 539, 109],[278, 539, 109],[277, 539, 109],
    [276, 539, 109],[275, 539, 109],[274, 539, 109],[285, 558, 109],[285, 557, 109],[285, 556, 109],[285, 555, 109],[284, 560, 110],
    [284, 559, 110],[284, 558, 110],[284, 557, 110],[284, 556, 110],[284, 555, 110],[284, 554, 110],[284, 553, 110],[283, 560, 110],
    [283, 559, 110],[283, 558, 110],[283, 557, 110],[283, 556, 110],[283, 555, 110],[283, 554, 110],[283, 553, 110],[62, 496, 80],
    [62, 495, 80],[62, 494, 80],[62, 493, 80],[67, 495, 81],[66, 495, 81],[65, 495, 81],[64, 495, 81],[63, 495, 81],[62, 495, 81]
]

positive_train67 = []
for x in range(90,430):
    for y in range(240,520):
        for z in range(25,40):
            if target67_3d_normalization[x,y,z] > 80:
                positive_train67.append([x,y,z])

In [ ]:
#S70TRP
positive_train70 = [
    [342, 408, 50],[343, 405, 51],[344, 402, 52],[390, 440, 53],[237, 433, 53],[345, 402, 53],[396, 436, 54],[349, 431, 54],[347, 396, 54],[472, 438, 54],[239, 434, 54],[395, 433, 55],[347, 435, 55],[135, 473, 55],[121, 481, 55],[385, 427, 56],[469, 451, 56],[238, 405, 56],[134, 472, 56],[120, 479, 56],[469, 438, 57],[386, 425, 57],[393, 424, 57],[236, 405, 57],[192, 446, 57],[132, 470, 57],[117, 473, 57],[471, 437, 58],[393, 423, 58],[344, 432, 58],[192, 444, 58],[131, 468, 58],[116, 472, 58],[232, 401, 58],[471, 434, 59],[394, 423, 59],[338, 429, 59],[200, 435, 59],[128, 465, 59],[115, 470, 59],[228, 397, 59],[472, 432, 60],[396, 422, 60],[328, 437, 60],[193, 438, 60],[225, 395, 60],[125, 459, 60],[114, 469, 60],[473, 431, 61],[397, 423, 61],[196, 435, 61],[125, 458, 61],[112, 466, 61],[474, 429, 62],[395, 425, 62],[327, 394, 62],[212, 433, 62],[196, 435, 62],[123, 450, 62],[123, 467, 62],[112, 466, 62],[482, 447, 63],[472, 478, 63],[473, 436, 63],[402, 422, 63],[211, 434, 63],[192, 434, 63],[120, 443, 63],[126, 468, 63],[112, 466, 63],[482, 447, 64],[474, 437, 64],[406, 419, 64],[210, 435, 64],[191, 435, 64],[118, 438, 64],[126, 469, 64],[111, 467, 64],[482, 446, 65],[412, 416, 65],[403, 426, 65],[209, 435, 65],[190, 434, 65],[117, 435, 65],[124, 466, 65],[110, 464, 65],[476, 470, 66],[483, 446, 66],[473, 427, 66],[416, 414, 66],[405, 425, 66],[207, 436, 66],[189, 434, 66],[116, 433, 66],[123, 468, 66],[108, 461, 66],[482, 444, 67],[479, 461, 67],[419, 412, 67],[410, 423, 67],[206, 437, 67],[188, 434, 67],[115, 431, 67],[122, 465, 67],[133, 486, 67],[479, 460, 68],[483, 445, 68],[473, 424, 68],[413, 422, 68],[204, 437, 68],[187, 433, 68],[115, 431, 68],[121, 464, 68],[107, 459, 68],[105, 474, 68],[473, 422, 69],[483, 444, 69],[481, 461, 69],[422, 410, 69],[416, 421, 69],[201, 438, 69],
    [185, 433, 69],[114, 425, 69],[106, 456, 69],[120, 464, 69],[105, 478, 69],[131, 485, 69],[482, 460, 70],[475, 426, 70],[483, 414, 70],[416, 421, 70],[423, 409, 70],[198, 438, 70],[182, 432, 70],[113, 424, 70],[119, 462, 70],[106, 456, 70],[482, 458, 71],[485, 448, 71],[476, 425, 71],[483, 414, 71],[418, 420, 71],[425, 408, 71],[195, 439, 71],[180, 432, 71],[113, 423, 71],[118, 463, 71],[105, 455, 71],[462, 489, 71],[440, 490, 71],[461, 490, 72],[439, 490, 72],[427, 406, 72],[420, 418, 72],[192, 440, 72],[178, 432, 72],[112, 419, 72],[117, 460, 72],[105, 455, 72],[123, 480, 72],[483, 457, 73],[482, 442, 73],[428, 406, 73],[420, 418, 73],[190, 441, 73],[175, 430, 73],[111, 418, 73],[104, 452, 73],[117, 460, 73],[117, 475, 73],[483, 442, 74],[482, 454, 74],[421, 417, 74],[429, 406, 74],[188, 442, 74],[174, 430, 74],[111, 417, 74],[104, 452, 74],[116, 459, 74],[118, 475, 74],[487, 411, 75],[422, 416, 75],[171, 429, 75],[187, 442, 75],[111, 415, 75],[103, 452, 75],[115, 457, 75],[116, 473, 75],[481, 452, 76],[484, 442, 76],[169, 428, 76],[100, 450, 76],[114, 457, 76],[114, 471, 76],[482, 454, 77],[168, 427, 77],[186, 442, 77],[100, 450, 77],[113, 454, 77],[113, 469, 77],[482, 455, 78],[485, 442, 78],[167, 426, 78],[423, 414, 78],[186, 443, 78],[99, 449, 78],[111, 454, 78],[112, 468, 78],[482, 456, 79],[479, 454, 79],[486, 443, 79],[186, 443, 79],[166, 425, 79],[110, 448, 79],[100, 449, 79],[110, 466, 79],[489, 462, 80],[481, 453, 80],[487, 443, 80],[186, 443, 80],[166, 424, 80],[109, 447, 80],[110, 464, 80],[98, 449, 80],[489, 458, 81],[480, 454, 81],[488, 443, 81],[186, 444, 81],[165, 422, 81],[109, 446, 81],[98, 448, 81],[109, 462, 81],[481, 454, 82],[489, 459, 82],[488, 444, 82],[500, 483, 82],[187, 444, 82],[165, 423, 82],[109, 445, 82],[108, 461, 82],[97, 447, 82],[480, 453, 83],[488, 462, 83],
    [488, 445, 83],[425, 412, 83],[187, 445, 83],[164, 422, 83],[109, 444, 83],[98, 448, 83],[106, 458, 83],[499, 481, 84],[492, 464, 84],[481, 455, 84],[489, 445, 84],[424, 411, 84],[188, 446, 84],[164, 421, 84],[108, 442, 84],[97, 448, 84],[106, 456, 84],[104, 475, 84],[499, 478, 85],[492, 465, 85],[480, 454, 85],[489, 445, 85],[424, 411, 85],[189, 447, 85],[164, 421, 85],[108, 443, 85],[97, 448, 85],[105, 455, 85],[495, 467, 86],[480, 454, 86],[489, 446, 86],[483, 476, 86],[424, 410, 86],[189, 447, 86],[105, 438, 86],[97, 448, 86],[105, 454, 86],[494, 468, 87],[481, 475, 87],[480, 455, 87],[489, 446, 87],[496, 437, 87],[424, 410, 87],[190, 448, 87],[163, 419, 87],[106, 438, 87],[96, 448, 87],[104, 453, 87],[500, 471, 88],[476, 477, 88],[480, 456, 88],[490, 446, 88],[496, 440, 88],[384, 444, 88],[192, 450, 88],[163, 419, 88],[106, 438, 88],[94, 448, 88],[104, 451, 88],[104, 481, 88],[479, 455, 89],[490, 445, 89],[423, 410, 89],[193, 451, 89],[163, 418, 89],[104, 436, 89],[102, 447, 89],[94, 449, 89],[103, 482, 89],[478, 456, 90],[494, 441, 90],[423, 409, 90],[194, 452, 90],[162, 418, 90],[104, 442, 90],[104, 431, 90],[94, 449, 90],[478, 457, 91],[489, 445, 91],[423, 408, 91],[197, 454, 91],[162, 418, 91],[101, 428, 91],[100, 443, 91],[93, 449, 91],[103, 484, 91],[476, 458, 92],[488, 445, 92],[423, 409, 92],[199, 456, 92],[162, 417, 92],[103, 427, 92],[92, 447, 92],[105, 440, 92],[103, 485, 92],[477, 460, 93],[488, 444, 93],[422, 409, 93],[399, 411, 93],[162, 417, 93],[162, 417, 93],[105, 440, 93],[99, 443, 93],[90, 449, 93],[102, 486, 93],[200, 456, 93],[476, 456, 94],[494, 446, 94],[422, 409, 94],[162, 416, 94],[106, 437, 94],[98, 444, 94],[90, 450, 94],[200, 457, 94],[101, 486, 94],[103, 422, 94],[475, 466, 95],[474, 457, 95],[494, 447, 95],[487, 443, 95],[397, 411, 95],[421, 409, 95],[162, 417, 95],
    [201, 458, 95],[100, 486, 95],[104, 496, 95],[91, 449, 95],[96, 442, 95],[106, 437, 95],[476, 457, 96],[475, 458, 96],[494, 447, 96],[486, 442, 96],[421, 409, 96],[163, 416, 96],[201, 458, 96],[90, 451, 96],[95, 442, 96],[106, 436, 96],[99, 485, 96],[475, 467, 97],[475, 455, 97],[495, 447, 97],[421, 408, 97],[202, 459, 97],[163, 416, 97],[108, 432, 97],[95, 441, 97],[90, 449, 97],[98, 486, 97],[473, 471, 98],[474, 454, 98],[495, 447, 98],[481, 434, 98],[420, 408, 98],[203, 460, 98],[163, 416, 98],[108, 431, 98],[95, 439, 98],[90, 450, 98],[97, 485, 98],[472, 471, 99],[473, 453, 99],[420, 408, 99],[204, 461, 99],[164, 415, 99],[109, 425, 99],[93, 439, 99],[90, 450, 99],[472, 472, 100],[470, 454, 100],[479, 432, 100],[496, 448, 100],[420, 409, 100],[164, 414, 100],[375, 455, 100],[93, 438, 100],[89, 451, 100],[471, 472, 101],[472, 453, 101],[419, 409, 101],[207, 464, 101],[165, 415, 101],[89, 452, 101],[91, 441, 101],[471, 472, 102],[472, 455, 102],[419, 409, 102],[473, 425, 102],[166, 415, 102],[469, 473, 103],[471, 453, 103],[472, 424, 103],[419, 409, 103],[167, 414, 103],[110, 414, 103],[470, 423, 104],[470, 452, 104],[468, 472, 104],[418, 407, 104],[168, 414, 104],[469, 422, 105],[418, 409, 105],[468, 451, 105],[169, 414, 105],[109, 411, 105],[467, 449, 106],[418, 409, 106],[170, 414, 106],[467, 450, 107],[418, 408, 107],[170, 414, 107],[463, 473, 108],[466, 448, 108],[465, 418, 108],[418, 408, 108],[171, 413, 108],[461, 476, 109],[466, 449, 109],[465, 417, 109],[417, 409, 109],[171, 413, 109],[464, 446, 110],[460, 476, 110],[465, 447, 110],[464, 416, 110],[416, 409, 110],[172, 412, 110],[108, 406, 110],[459, 474, 111],[464, 445, 111],[464, 416, 111],[416, 409, 111],[172, 411, 111],[465, 446, 112],[463, 415, 112],[415, 409, 112],[173, 411, 112],[465, 444, 113],[462, 414, 113],[414, 409, 113],[323, 403, 113]
]

positive_train70 = []
for x in range(150,240):
    for y in range(270,400):
        for z in range(25,40):
            if target70_3d_normalization[x,y,z] > 80:
                positive_train70.append([x,y,z])

In [ ]:
#S77TRP
positive_train77 = [
    [347, 400, 36],[346, 400, 36],[337, 394, 36],[336, 394, 36],[335, 394, 36],[265, 406, 36],[264, 406, 36],[263, 406, 36],[347, 398, 37],[346, 398, 37],[337, 393, 37],[336, 393, 37],[335, 393, 37],[265, 404, 37],[264, 404, 37],[263, 404, 37],[262, 404, 37],[347, 398, 38],[346, 398, 38],[337, 390, 38],[336, 390, 38],[335, 390, 38],[264, 403, 38],[263, 403, 38],[262, 403, 38],[261, 403, 38],[378, 405, 38],[374, 405, 38],[347, 397, 39],[346, 397, 39],[337, 388, 39],[336, 388, 39],[335, 388, 39],[264, 401, 39],[262, 401, 39],[261, 401, 39],[260, 418, 39],[259, 418, 39],[219, 425, 39],[218, 425, 39],[371, 404, 40],[370, 404, 40],[369, 404, 40],[357, 408, 40],[356, 408, 40],[347, 397, 40],[346, 397, 40],[337, 386, 40],[336, 386, 40],[335, 386, 40],[263, 398, 40],[261, 398, 40],[260, 398, 40],[260, 416, 40],[271, 386, 40],[270, 386, 40],[269, 386, 40],[261, 386, 40],[329, 378, 40],[328, 378, 40],[325, 378, 40],[322, 378, 40],[357, 407, 41],[356, 407, 41],[348, 396, 41],[347, 396, 41],[346, 396, 41],[337, 384, 41],[336, 384, 41],[335, 384, 41],[328, 375, 41],[323, 375, 41],[322, 375, 41],[320, 375, 41],[272, 384, 41],[271, 384, 41],[270, 384, 41],[262, 384, 41],[262, 396, 41],[260, 396, 41],[259, 396, 41],[261, 414, 41],[260, 414, 41],[220, 423, 41],[219, 423, 41],[348, 397, 42],[347, 397, 42],[338, 383, 42],[337, 383, 42],[336, 383, 42],[327, 375, 42],[322, 375, 42],[320, 375, 42],[319, 375, 42],[273, 385, 42],[271, 385, 42],[263, 385, 42],[261, 395, 42],[259, 395, 42],[258, 395, 42],[261, 411, 42],[260, 411, 42],[239, 418, 42],[222, 422, 42],[220, 422, 42],[461, 417, 43],[357, 406, 43],[356, 406, 43],[350, 397, 43],[349, 397, 43],[348, 397, 43],[338, 381, 43],[339, 381, 43],[317, 375, 43],[326, 375, 43],[324, 375, 43],[320, 375, 43],[319, 375, 43],[259, 391, 43],[257, 391, 43],[255, 391, 43],[240, 420, 43],[222, 422, 43],[221, 422, 43],[463, 413, 44],[466, 427, 44],[462, 427, 44],[390, 413, 44],[391, 413, 44],[389, 413, 44],[386, 413, 44],[385, 413, 44],[367, 410, 44],[366, 410, 44],[356, 403, 44],[355, 403, 44],[338, 379, 44],[339, 379, 44],[337, 379, 44],[255, 389, 44],[254, 389, 44],[232, 422, 44],[233, 422, 44],[230, 426, 44],[228, 426, 44],[463, 415, 45],[465, 426, 45],[466, 426, 45],[391, 410, 45],[368, 412, 45],[367, 412, 45],[352, 399, 45],[351, 399, 45],[254, 388, 45],[255, 388, 45],[253, 388, 45],[339, 375, 45],[340, 375, 45],[338, 375, 45],[224, 425, 45],[212, 429, 45],[211, 429, 45],[464, 413, 46],[470, 413, 46],[462, 423, 46],[392, 409, 46],[393, 409, 46],[391, 409, 46],[386, 412, 46],[387, 412, 46],[384, 412, 46],[340, 374, 46],[341, 374, 46],[339, 374, 46],[253, 386, 46],[254, 386, 46],[251, 386, 46],[233, 426, 46],[224, 425, 46],[223, 425, 46],[212, 428, 46],[213, 428, 46],[481, 439, 47],[484, 439, 47],[479, 439, 47],[464, 412, 47],[462, 412, 47],[461, 423, 47],[465, 423, 47],[394, 407, 47],[393, 407, 47],[386, 410, 47],[385, 410, 47],[371, 415, 47],[341, 373, 47],[342, 373, 47],[340, 373, 47],[251, 385, 47],[252, 385, 47],[250, 385, 47],[267, 434, 47],[266, 434, 47],[246, 427, 47],[247, 427, 47],[234, 430, 47],[235, 430, 47],[224, 426, 47],[223, 426, 47],[212, 427, 47],[213, 427, 47],[211, 427, 47],[145, 442, 47],[147, 442, 47],[463, 413, 48],[462, 413, 48],[460, 421, 48],[396, 407, 48],[394, 407, 48],[387, 412, 48],[388, 412, 48],[385, 412, 48],[386, 412, 48],[342, 372, 48],[343, 372, 48],[341, 372, 48],[250, 384, 48],[251, 384, 48],[248, 384, 48],[235, 433, 48],[234, 433, 48],
    [223, 430, 48],[222, 430, 48],[212, 428, 48],[147, 442, 48],[466, 410, 49],[463, 410, 49],[460, 420, 49],[464, 420, 49],[458, 420, 49],[398, 406, 49],[396, 406, 49],[388, 411, 49],[389, 411, 49],[387, 411, 49],[343, 370, 49],[344, 370, 49],[342, 370, 49],[248, 382, 49],[249, 382, 49],[246, 382, 49],[234, 435, 49],[233, 435, 49],[222, 428, 49],[223, 428, 49],[212, 426, 49],[211, 426, 49],[144, 439, 49],[147, 439, 49],[464, 409, 50],[463, 419, 50],[464, 419, 50],[398, 406, 50],[397, 406, 50],[390, 412, 50],[391, 412, 50],[387, 412, 50],[379, 416, 50],[344, 370, 50],[346, 370, 50],[343, 370, 50],[246, 381, 50],[248, 381, 50],[247, 381, 50],[233, 435, 50],[221, 429, 50],[222, 429, 50],[211, 427, 50],[212, 427, 50],[210, 427, 50],[146, 437, 50],[147, 437, 50],[142, 437, 50],[467, 407, 51],[465, 407, 51],[472, 407, 51],[458, 417, 51],[463, 417, 51],[400, 405, 51],[401, 405, 51],[399, 405, 51],[390, 411, 51],[389, 411, 51],[346, 368, 51],[348, 368, 51],[345, 368, 51],[344, 368, 51],[244, 380, 51],[246, 380, 51],[230, 437, 51],[229, 437, 51],[220, 429, 51],[221, 429, 51],[210, 427, 51],[211, 427, 51],[143, 435, 51],[146, 435, 51],[465, 408, 52],[462, 415, 52],[463, 415, 52],[401, 405, 52],[402, 405, 52],[400, 405, 52],[393, 413, 52],[348, 368, 52],[346, 368, 52],[241, 378, 52],[242, 378, 52],[228, 438, 52],[219, 430, 52],[220, 430, 52],[218, 430, 52],[210, 428, 52],[211, 428, 52],[208, 428, 52],[145, 434, 52],[146, 434, 52],[466, 408, 53],[465, 408, 53],[461, 414, 53],[462, 414, 53],[403, 404, 53],[404, 404, 53],[401, 404, 53],[394, 412, 53],[393, 412, 53],[236, 379, 53],[240, 379, 53],[226, 439, 53],[218, 430, 53],[219, 430, 53],[208, 428, 53],[207, 428, 53],[210, 428, 53],[145, 433, 53],[470, 404, 54],[458, 410, 54],[456, 410, 54],[403, 404, 54],[405, 404, 54],[398, 413, 54],[395, 413, 54],[386, 422, 54],[231, 379, 54],[216, 431, 54],[217, 431, 54],[214, 438, 54],[207, 429, 54],[208, 429, 54],[206, 429, 54],[144, 431, 54],[145, 431, 54],[484, 408, 55],[467, 406, 55],[466, 406, 55],[399, 413, 55],[398, 413, 55],[405, 402, 55],[406, 402, 55],[404, 402, 55],[362, 360, 55],[223, 377, 55],[223, 441, 55],[212, 438, 55],[211, 438, 55],[214, 431, 55],[215, 431, 55],[206, 428, 55],[207, 428, 55],[205, 428, 55],[140, 430, 55],[144, 430, 55],[468, 404, 56],[467, 404, 56],[474, 404, 56],[460, 408, 56],[461, 408, 56],[407, 411, 56],[406, 401, 56],[407, 401, 56],[405, 401, 56],[220, 374, 56],[212, 430, 56],[210, 430, 56],[205, 428, 56],[206, 428, 56],[204, 428, 56],[142, 428, 56],[143, 428, 56],[488, 405, 57],[487, 405, 57],[476, 421, 57],[477, 421, 57],[474, 403, 57],[468, 403, 57],[475, 403, 57],[460, 407, 57],[461, 407, 57],[412, 406, 57],[413, 406, 57],[411, 406, 57],[409, 399, 57],[210, 430, 57],[204, 428, 57],[205, 428, 57],[203, 428, 57],[208, 440, 57],[142, 428, 57],[143, 428, 57],[477, 419, 58],[474, 402, 58],[475, 402, 58],[468, 402, 58],[457, 405, 58],[460, 405, 58],[412, 406, 58],[411, 397, 58],[208, 429, 58],[203, 428, 58],[202, 428, 58],[138, 425, 58],[142, 425, 58],[477, 416, 59],[478, 416, 59],[470, 402, 59],[468, 402, 59],[476, 402, 59],[457, 403, 59],[460, 403, 59],[455, 403, 59],[413, 404, 59],[414, 404, 59],[413, 394, 59],[201, 427, 59],[137, 424, 59],[141, 424, 59],[135, 424, 59],[470, 401, 60],[476, 401, 60],[459, 403, 60],[460, 403, 60],[455, 403, 60],[413, 403, 60],[414, 403, 60],[415, 391, 60],[414, 391, 60],[205, 428, 60],[206, 428, 60],[199, 425, 60],[200, 425, 60],[139, 423, 60],
    [140, 423, 60],[134, 423, 60],[470, 401, 61],[469, 401, 61],[477, 401, 61],[459, 401, 61],[460, 401, 61],[455, 401, 61],[414, 404, 61],[416, 388, 61],[415, 388, 61],[195, 423, 61],[138, 423, 61],[470, 400, 62],[469, 400, 62],[477, 400, 62],[459, 401, 62],[460, 401, 62],[455, 401, 62],[413, 403, 62],[414, 403, 62],[416, 386, 62],[392, 417, 62],[393, 417, 62],[201, 427, 62],[190, 420, 62],[137, 420, 62],[138, 420, 62],[472, 400, 63],[477, 400, 63],[459, 400, 63],[460, 400, 63],[413, 403, 63],[414, 403, 63],[416, 384, 63],[200, 427, 63],[201, 427, 63],[185, 416, 63],[137, 420, 63],[472, 400, 64],[470, 400, 64],[477, 400, 64],[456, 398, 64],[460, 398, 64],[459, 398, 64],[412, 403, 64],[413, 403, 64],[411, 403, 64],[416, 382, 64],[415, 382, 64],[393, 418, 64],[392, 418, 64],[199, 426, 64],[200, 426, 64],[184, 416, 64],[183, 416, 64],[135, 418, 64],[136, 418, 64],[473, 400, 65],[478, 400, 65],[470, 400, 65],[459, 397, 65],[460, 397, 65],[412, 403, 65],[411, 403, 65],[413, 403, 65],[415, 381, 65],[416, 381, 65],[392, 418, 65],[393, 418, 65],[198, 426, 65],[199, 426, 65],[182, 414, 65],[183, 414, 65],[134, 417, 65],[135, 417, 65],[129, 417, 65],[473, 398, 66],[471, 398, 66],[478, 398, 66],[459, 395, 66],[460, 395, 66],[454, 395, 66],[412, 403, 66],[415, 380, 66],[416, 380, 66],[198, 426, 66],[199, 426, 66],[182, 415, 66],[183, 415, 66],[181, 415, 66],[129, 417, 66],[134, 417, 66],[128, 417, 66],[473, 400, 67],[477, 400, 67],[470, 400, 67],[471, 400, 67],[459, 396, 67],[460, 396, 67],[455, 396, 67],[411, 402, 67],[412, 402, 67],[415, 380, 67],[416, 380, 67],[198, 426, 67],[199, 426, 67],[182, 415, 67],[183, 415, 67],[181, 415, 67],[132, 416, 67],[133, 416, 67],[126, 416, 67],[473, 400, 68],[477, 400, 68],[471, 400, 68],[478, 390, 68],[474, 390, 68],[459, 395, 68],[460, 395, 68],[410, 401, 68],[411, 401, 68],[415, 379, 68],[416, 379, 68],[199, 425, 68],[198, 425, 68],[182, 415, 68],[183, 415, 68],[129, 414, 68],[127, 414, 68],[131, 414, 68],[474, 400, 69],[476, 400, 69],[471, 400, 69],[478, 387, 69],[459, 393, 69],[460, 393, 69],[455, 393, 69],[409, 401, 69],[410, 401, 69],[199, 425, 69],[200, 425, 69],[182, 415, 69],[183, 415, 69],[129, 415, 69],[131, 415, 69],[125, 415, 69],[466, 443, 70],[473, 400, 70],[476, 400, 70],[470, 400, 70],[459, 392, 70],[460, 392, 70],[476, 386, 70],[411, 403, 70],[410, 403, 70],[415, 378, 70],[200, 425, 70],[199, 425, 70],[183, 414, 70],[182, 414, 70],[184, 414, 70],[135, 421, 70],[136, 421, 70],[132, 421, 70],[128, 438, 70],[130, 438, 70],[125, 438, 70],[129, 415, 70],[130, 415, 70],[125, 415, 70],[473, 400, 71],[476, 400, 71],[470, 400, 71],[471, 400, 71],[476, 386, 71],[459, 392, 71],[460, 392, 71],[455, 392, 71],[411, 403, 71],[415, 377, 71],[201, 426, 71],[200, 426, 71],[183, 414, 71],[184, 414, 71],[135, 418, 71],[136, 418, 71],[128, 416, 71],[129, 416, 71],[123, 416, 71],[127, 426, 71],[128, 426, 71],[126, 426, 71],[471, 399, 72],[476, 399, 72],[470, 399, 72],[476, 383, 72],[480, 383, 72],[473, 383, 72],[459, 390, 72],[460, 390, 72],[455, 390, 72],[412, 404, 72],[415, 377, 72],[202, 426, 72],[201, 426, 72],[184, 414, 72],[183, 414, 72],[127, 416, 72],[122, 416, 72],[126, 440, 72],[128, 440, 72],[127, 440, 72],[472, 399, 73],[469, 399, 73],[475, 399, 73],[473, 381, 73],[472, 381, 73],[459, 389, 73],[460, 389, 73],[455, 389, 73],[407, 405, 73],[415, 376, 73],[414, 376, 73],[202, 426, 73],[185, 415, 73],[186, 415, 73],[126, 415, 73],[128, 415, 73],[126, 440, 73],
    [127, 440, 73],[122, 440, 73],[472, 399, 74],[475, 378, 74],[460, 390, 74],[410, 405, 74],[414, 375, 74],[186, 417, 74],[203, 427, 74],[135, 412, 74],[125, 440, 74],[471, 399, 75],[473, 377, 75],[459, 387, 75],[410, 405, 75],[413, 374, 75],[204, 428, 75],[187, 416, 75],[135, 412, 75],[125, 416, 75],[123, 442, 75],[471, 398, 76],[472, 378, 76],[460, 388, 76],[407, 407, 76],[412, 373, 76],[204, 429, 76],[187, 416, 76],[135, 411, 76],[124, 441, 76],[123, 416, 76],[471, 398, 77],[473, 374, 77],[460, 386, 77],[409, 406, 77],[411, 373, 77],[205, 430, 77],[188, 415, 77],[124, 440, 77],[134, 407, 77],[122, 416, 77],[470, 397, 78],[472, 373, 78],[460, 387, 78],[409, 406, 78],[411, 373, 78],[206, 431, 78],[188, 416, 78],[123, 441, 78],[134, 408, 78],[122, 417, 78],[470, 397, 79],[472, 371, 79],[459, 384, 79],[411, 373, 79],[206, 431, 79],[188, 415, 79],[133, 405, 79],[122, 417, 79],[123, 440, 79],[469, 395, 80],[473, 374, 80],[460, 384, 80],[411, 373, 80],[409, 406, 80],[207, 432, 80],[188, 414, 80],[133, 404, 80],[121, 417, 80],[470, 396, 81],[460, 383, 81],[471, 372, 81],[407, 412, 81],[188, 414, 81],[133, 404, 81],[120, 417, 81],[133, 405, 81],[470, 395, 82],[472, 370, 82],[460, 383, 82],[412, 372, 82],[209, 433, 82],[188, 414, 82],[133, 404, 82],[119, 417, 82],[469, 393, 83],[468, 378, 83],[472, 369, 83],[458, 379, 83],[409, 407, 83],[412, 372, 83],[210, 434, 83],[188, 413, 83],[127, 386, 83],[132, 403, 83],[119, 416, 83],[124, 436, 83],[117, 433, 83],[466, 387, 84],[474, 364, 84],[458, 376, 84],[464, 411, 84],[409, 408, 84],[188, 412, 84],[125, 435, 84],[118, 433, 84],[118, 417, 84],[132, 401, 84],[127, 383, 84],[459, 391, 85],[460, 380, 85],[475, 366, 85],[408, 408, 85],[187, 412, 85],[126, 432, 85],[120, 432, 85],[117, 418, 85],[132, 401, 85],[128, 377, 85],[467, 387, 86],[458, 375, 86],[473, 366, 86],[408, 409, 86],[188, 412, 86],[127, 430, 86],[121, 432, 86],[117, 417, 86],[129, 400, 86],[129, 371, 86],[466, 384, 87],[458, 391, 87],[458, 372, 87],[476, 365, 87],[413, 370, 87],[188, 412, 87],[128, 426, 87],[122, 433, 87],[116, 419, 87],[131, 400, 87],[130, 368, 87],[459, 393, 88],[466, 384, 88],[459, 374, 88],[476, 364, 88],[406, 410, 88],[412, 369, 88],[189, 412, 88],[127, 428, 88],[123, 433, 88],[113, 419, 88],[130, 397, 88],[131, 363, 88],[459, 393, 89],[466, 384, 89],[457, 371, 89],[477, 363, 89],[410, 369, 89],[406, 410, 89],[215, 439, 89],[190, 411, 89],[128, 423, 89],[125, 433, 89],[114, 419, 89],[130, 398, 89],[459, 393, 90],[466, 381, 90],[457, 371, 90],[406, 411, 90],[408, 370, 90],[217, 441, 90],[191, 411, 90],[129, 423, 90],[113, 418, 90],[126, 397, 90],[133, 436, 90],[459, 393, 91],[465, 377, 91],[457, 369, 91],[407, 370, 91],[478, 379, 91],[406, 412, 91],[218, 441, 91],[191, 411, 91],[136, 437, 91],[129, 421, 91],[113, 419, 91],[130, 395, 91],[459, 393, 92],[475, 378, 92],[465, 374, 92],[458, 367, 92],[407, 370, 92],[192, 411, 92],[133, 437, 92],[130, 416, 92],[109, 420, 92],[130, 395, 92],[458, 392, 93],[473, 377, 93],[463, 368, 93],[457, 367, 93],[406, 370, 93],[192, 412, 93],[111, 419, 93],[132, 413, 93],[129, 394, 93],[458, 392, 94],[471, 376, 94],[465, 367, 94],[457, 364, 94],[406, 370, 94],[193, 412, 94],[108, 420, 94],[133, 411, 94],[129, 393, 94],[458, 390, 95],[469, 376, 95],[466, 364, 95],[458, 363, 95],[405, 414, 95],[193, 413, 95],[107, 419, 95],[132, 405, 95],[129, 392, 95],[137, 356, 95],[458, 390, 96],[467, 377, 96],[459, 364, 96],[193, 413, 96],[106, 420, 96]
]

positive_train77 = []
for x in range(110,400):
    for y in range(250,530):
        for z in range(40,60):
            if target77_3d_normalization[x,y,z] > 80:
                positive_train77.append([x,y,z])

In [ ]:
#S65TRP
positive_train65 = [
    [382, 438, 61],[383, 438, 61],[383, 438, 62],[385, 437, 62],[384, 437, 62],[383, 437, 62],[384, 437, 63],[383, 437, 63],[382, 437, 63],
    [383, 437, 64],[383, 436, 64],[383, 435, 64],[383, 434, 64],[384, 436, 64],[384, 435, 64],[384, 434, 64],[384, 435, 65],[384, 434, 65],
    [384, 433, 65],[384, 433, 66],[384, 432, 66],[383, 433, 66],[383, 432, 66],[355, 478, 53],[355, 477, 53],[356, 476, 53],[357, 477, 53],
    [356, 477, 54],[356, 476, 54],[357, 477, 54],[357, 476, 54],[356, 476, 55],[356, 476, 56],[356, 475, 56],[357, 476, 56],[357, 475, 56],
    [356, 475, 57],[357, 476, 57],[357, 475, 57],[356, 475, 58],[357, 475, 58],[357, 474, 59],[356, 474, 60],[357, 474, 60],[355, 474, 61],
    [356, 474, 61],[354, 474, 62],[354, 473, 62],[355, 474, 62],[355, 473, 62],[353, 474, 63],[354, 474, 63],[354, 473, 63],[355, 474, 63],
    [355, 473, 63],[352, 474, 64],[352, 473, 64],[351, 474, 64],[351, 473, 64],[352, 474, 64],[352, 473, 64],[353, 474, 64],[353, 473, 64],
    [354, 474, 64],[354, 473, 64],[355, 474, 64],[356, 474, 64],[350, 474, 65],[350, 473, 65],[351, 474, 65],[351, 473, 65],[352, 474, 65],
    [352, 473, 65],[353, 474, 65],[353, 473, 65],[354, 474, 65],[351, 475, 66],[350, 475, 66],[349, 475, 66],[352, 474, 66],[351, 474, 66],
    [350, 474, 66],[349, 474, 66],[349, 475, 67],[348, 475, 67],[347, 475, 67],[367, 477, 52],[368, 477, 53],[368, 478, 53],[369, 478, 53],
    [369, 478, 54],[370, 479, 54],[370, 478, 54],[370, 479, 55],[371, 479, 55],[371, 480, 55],[371, 479, 55],[372, 480, 55],[372, 479, 55],
    [371, 480, 56],[372, 480, 56],[371, 481, 57],[372, 481, 57],[371, 482, 58],[371, 481, 58],[372, 482, 58],[372, 481, 58],[371, 483, 59],
    [371, 482, 59],[372, 483, 59],[372, 482, 59],[371, 483, 60],[372, 483, 60],[372, 482, 60],[371, 484, 61],[371, 483, 61],[372, 484, 61],
    [372, 483, 61],[370, 484, 62],[370, 483, 62],[371, 484, 62],[371, 483, 62],[372, 484, 62],[372, 483, 62],[370, 484, 62],[370, 483, 62],
    [370, 484, 63],[370, 483, 63],[370, 484, 64],[369, 484, 64],[369, 484, 65],[368, 485, 66],[368, 484, 66],[369, 485, 66],[369, 484, 66],
    [370, 485, 66],[370, 484, 66],[368, 485, 67],[368, 484, 67],[366, 486, 68],[366, 485, 68],[367, 486, 68],[367, 485, 68],[365, 487, 70],
    [365, 486, 70],[366, 487, 70],[366, 486, 70],[363, 487, 71],[364, 487, 71],[364, 486, 71],[364, 488, 71],[363, 488, 71],[363, 486, 71],
    [363, 488, 72],[363, 487, 72],[363, 486, 72],[363, 485, 72],[364, 488, 72],[364, 487, 72],[364, 486, 72],[362, 488, 73],[363, 489, 73],
    [363, 488, 73],[363, 487, 73],[363, 486, 73],[363, 485, 73],[369, 456, 70],[369, 455, 70],[369, 454, 70],[369, 453, 70],[370, 456, 70],
    [370, 455, 70],[370, 454, 70],[370, 453, 70],[368, 457, 71],[369, 457, 71],[369, 456, 71],[369, 458, 71],[370, 457, 71],[370, 456, 71],
    [370, 455, 71],[370, 454, 71],[367, 458, 72],[367, 457, 72],[368, 458, 72],[368, 457, 72],[368, 456, 72],[369, 458, 72],[369, 457, 72],
]

# y_train_1 = np.ones((len(positive_train)+len(positive_train65)+len(positive_train67)+len(positive_train70)+len(positive_train77)))

In [ ]:
#S57TRN
negative_train = [
[331,522,38],[321,444,23],[337,466,23],[366,488,23],[381,474,23],[393,500,19],[388,498,19],[382,474,20],[410,495,19],[422,528,24],
[336,489,38],[443,480,24],[412,454,24],[270,416,20],[283,414,20],[332,405,23],[368,381,16],[356,376,16],[374,356,19],[361,352,19],
[485,338,19],[465,353,19],[328,381,16],[230,364,16],[226,337,16],[333,453,24],[244,440,29],[244,433,30],[216,416,30],[232,404,30],
[219,408,30],[240,409,30],[291,403,34],[322,408,29],[362,463,30],[362,461,30],[362,459,30],[364,459,30],[364,461,30],[364,463,30],
[366,463,30],[366,461,30],[366,459,30],[216,416,30],[238,409,30],[237,426,30],[238,436,30],[255,430,30],[88,448,52],[94,499,58],
    [324, 432, 26],[331, 432, 26],[332, 432, 26],[327, 431, 26],[331, 431, 26],[332, 431, 26],[327, 430, 26],[336, 430, 26],[331, 430, 26],[328, 429, 26],
    [340, 463, 27],[337, 463, 27],[336, 463, 27],[335, 462, 27],[331, 462, 27],[343, 462, 27],[334, 461, 27],[342, 460, 27],[326, 460, 27],[328, 458, 27],
    [341, 457, 27],[341, 456, 27],[329, 456, 27],[341, 455, 27],[341, 454, 27],[340, 449, 27],[339, 448, 27],[326, 448, 27],[343, 448, 27],[327, 446, 27],
    [343, 446, 27],[335, 446, 27],[342, 445, 27],[338, 444, 27],[340, 443, 27],[339, 442, 27],[340, 441, 27],[338, 440, 27],[339, 439, 27],[339, 438, 27],
    [340, 437, 27],[342, 437, 27],[328, 437, 27],[325, 437, 27],[324, 437, 27],[326, 436, 27],[341, 435, 27],[338, 434, 27],[337, 433, 27],[341, 433, 27],
    [338, 432, 27],[337, 432, 27],[326, 431, 27],[340, 431, 27],[326, 430, 27],[340, 430, 27],[326, 429, 27],[343, 429, 27],[342, 428, 27],[339, 428, 27],
    [326, 428, 27],[338, 427, 27],[342, 427, 27],[326, 426, 27],[339, 425, 27],[330, 448, 28],[342, 448, 28],[339, 447, 28],[329, 447, 28],[342, 446, 28],
    [330, 446, 28],[329, 446, 28],[341, 445, 28],[336, 445, 28],[328, 437, 28],[326, 430, 28],[338, 429, 28],[326, 428, 28],[340, 428, 28],[325, 427, 28],
[341, 465, 20],[337, 465, 20],[336, 465, 20],[335, 465, 20],[337, 464, 20],[336, 464, 20],[335, 464, 20],[337, 463, 20],[336, 463, 20],[335, 463, 20],
    [341, 425, 29],[339, 424, 29],[326, 424, 29],[326, 423, 29],[338, 423, 29],[342, 423, 29],[341, 422, 29],[342, 422, 29],[329, 422, 29],[324, 422, 29],
    [329, 421, 29],[368, 466, 29],[330, 421, 29],[340, 421, 29],[378, 473, 29],[378, 472, 29],[361, 467, 2],[362, 467, 29],[367, 466, 29],[361, 466, 29],
    [361, 465, 20],[367, 465, 31],[361, 458, 28],[360, 458, 28],[367, 458, 29],[360, 458, 29],[366, 457, 29],[360, 457, 29],[360, 457, 30],[366, 458, 30],
[341, 462, 20],[340, 462, 20],[336, 462, 20],[335, 462, 20],[341, 461, 20],[340, 461, 20],[341, 460, 20],[340, 460, 20],[335, 460, 20],[341, 459, 20],
[340, 459, 20],[336, 459, 20],[335, 459, 20],[341, 458, 20],[340, 458, 20],[336, 458, 20],[335, 458, 20],[340, 457, 21],[339, 457, 21],[335, 457, 21],
[334, 457, 21],[340, 456, 21],[339, 456, 21],[335, 456, 21],[334, 456, 21],[340, 455, 21],[339, 455, 21],[335, 455, 21],[334, 455, 21],[340, 454, 22],
[339, 454, 22],[335, 454, 22],[334, 454, 22],[339, 453, 22],[335, 453, 22],[334, 453, 22],[333, 453, 22],[341, 452, 23],[340, 452, 23],[339, 452, 23],
[338, 452, 23],[334, 452, 23],[333, 452, 23],[332, 452, 23],[331, 452, 23],[341, 451, 23],[340, 451, 23],[339, 451, 23],[338, 451, 23],[334, 451, 2],
[333, 451, 23],[341, 452, 24],[340, 452, 24],[337, 452, 24],[333, 452, 24],[332, 452, 24],[331, 452, 24],[341, 451, 24],[340, 451, 24],[337, 451, 24],
[333, 451, 24],[332, 451, 24],[331, 451, 24],[341, 450, 24],[340, 450, 24],[339, 450, 24],[338, 450, 24],[337, 450, 24],[337, 449, 24],[339, 449, 24],
[338, 449, 24],[341, 449, 24],[332, 449, 24],[331, 449, 24],[337, 448, 24],[336, 448, 24],[332, 448, 24],[331, 448, 24],[336, 447, 24],[332, 447, 24],
[331, 446, 24],[335, 446, 24],[331, 445, 24],[335, 445, 24],[337, 453, 25],[340, 449, 25],[336, 449, 25],[335, 449, 25],[332, 449, 25],[331, 449, 25],
[340, 448, 25],[336, 448, 25],[335, 448, 25],[331, 448, 25],[330, 448, 25],[330, 447, 25],[335, 447, 25],[336, 447, 25],[341, 447, 25],[342, 447, 25],
[325, 446, 25],[326, 446, 25],[329, 446, 25],[330, 446, 25],[335, 446, 25],[334, 445, 25],[329, 445, 25],[328, 445, 25],[325, 445, 25],[325, 444, 25],
[327, 444, 25],[334, 444, 25],[329, 444, 25],[333, 443, 25],[329, 443, 25],[333, 442, 25],[329, 442, 25],[328, 441, 25],[327, 441, 25],[332, 441, 25],
[334, 441, 25],[328, 440, 25],[332, 440, 25],[333, 440, 25],[327, 439, 25],[332, 439, 25],[336, 439, 25],[332, 438, 25],[328, 438, 25],[327, 438, 25],
[328, 437, 25],[331, 437, 25],[332, 437, 25],[327, 437, 25],[331, 436, 25],[327, 436, 25],[331, 435, 25],[327, 435, 25],[332, 434, 25],[327, 434, 25],
[322, 433, 26]
]
semble = negative_train
for n in range(len(semble)):
    x,y,z = semble[n]
    semble[n] = [int(round(x*insert0)),int(round(y*insert1)),int(round(z*insert2))]
a1,a2,b1,b2,c1,c2 = 180,220,490,500,27,34
a1 = int(a1*insert0)
a2 = int(a2*insert0)
b1 = int(b1*insert1)
b2 = int(b2*insert1)
c1 = int(c1*insert2)
c2 = int(c2*insert2)
for x in range(a1,a2,3):
    for y in range(b1,b2,3):
        for z in range(c1,c2,3):
#             x=int(round(x*insert0))
#             y=int(round(y*insert1))
#             z=int(round(z*insert2))
            if target_3d_normalization[x,y,z]<30:
                negative_train.append([x,y,z])
a1,a2,b1,b2,c1,c2 = 363,448,405,443,27,32
a1 = int(a1*insert0)
a2 = int(a2*insert0)
b1 = int(b1*insert1)
b2 = int(b2*insert1)
c1 = int(c1*insert2)
c2 = int(c2*insert2)               
for x in range(a1,a2,4):
    for y in range(b1,b2,4):
        for z in range(c1,c2,4):
#             x=int(round(x*insert0))
#             y=int(round(y*insert1))
#             z=int(round(z*insert2))
            if target_3d_normalization[x,y,z]<30:
                negative_train.append([x,y,z])


In [ ]:
#S67TRN
negative_train67 = [
    [240, 388, 29],[257,388,13],[213,384,14],[266,390,24],[290, 381, 30],[290, 380, 30],[290, 366, 30],[290, 375, 31],[290, 377, 32],
    [290, 376, 32],[290, 375, 32],[290, 362, 32],[290, 359, 32],[290, 383, 13],[290, 382, 13],[290, 371, 13],[290, 366, 13],[290, 382, 15],
    [290, 376, 15],[290, 375, 17],[296, 380, 29],[295, 380, 29],[301,382,19],[313,382,38],[291, 381, 30],[291, 371, 30],[291, 363, 30],
    [291, 380, 30],[291, 375, 30],[291, 374, 30] ,[291, 373, 30],[296, 368, 23],[296, 382, 24],[296, 381, 24],[296, 380, 24],[296, 375, 24],
    [296, 374, 24],[296, 373, 24],[296, 382, 25],[296, 381, 25],[296, 380, 25],[296, 379, 25],[296, 375, 25],[296, 374, 25],[296, 363, 25],
    [296, 380, 26],[296, 379, 26],[290, 384, 33],[290, 383, 33],[290, 382, 33],[290, 381, 33],[290, 379, 33],[290, 357, 33],[290, 380, 36],
    [290, 379, 36],[290, 366, 39],[66, 492, 83],[58,495,50],[55, 461, 70],[55, 460, 70],[55, 459, 70],[55, 457, 70],[90, 479, 50],
    [90, 472, 50],[90, 471, 50],[90, 464, 50],[90, 461, 50],[100, 489, 90],[100, 479, 100],[100, 460, 86],[379,461,25],[379,461,18],
    [332,461,27],[360,461,55],[343, 461, 30],[342, 461, 30],[341, 461, 30],[340, 461, 30],[339, 461, 30],[338, 461, 30],[367, 472, 35],
    [367, 470, 35],[367, 467, 35],[367, 466, 35],[277, 500, 56],[277, 494, 54],[277, 493, 54] ,[277, 485, 54],[279, 487, 56],[279, 486, 56],
    [279, 473, 56],[279, 472, 56],[279, 471, 56],[279, 469, 56],[279, 468, 56],[279, 467, 56],[279, 466, 56],[279, 465, 56],[279, 464, 56],
    [269, 480, 56],[269, 479, 56],[269, 478, 56],[269, 476, 56],[269, 475, 56],[269, 474, 56],[280, 543, 110],[279, 543, 110],[272, 543, 110],
    [271, 543, 110],[267, 539, 109],[262, 539, 109],[254, 539, 109],[257, 534, 109],[256, 534, 109],[255, 534, 109],[254, 534, 109],
    [51, 465, 70],[47, 465, 70],[46, 465, 70],[45, 465, 70],[55, 450, 81],[55, 449, 81],[55, 448, 81],[55, 447, 81],[55, 446, 81]
]
semble = negative_train67
for n in range(len(semble)):
    x,y,z = semble[n]
    semble[n] = [int(round((x+0.5)*insert0-1)),int(round((y+0.5)*insert1-1)),int(round((z+0.5)*insert2-1))]
a1,a2,b1,b2,c1,c2 = 330,415,420,475,61,65
a1 = int(a1*insert0)
a2 = int(a2*insert0)
b1 = int(b1*insert1)
b2 = int(b2*insert1)
c1 = int(c1*insert2)
c2 = int(c2*insert2)               
for x in range(a1,a2,4):
    for y in range(b1,b2,4):
        for z in range(c1,c2,4):
            if target67_3d_normalization[x,y,z]<30:
                negative_train67.append([x,y,z])

In [ ]:
#S70TRN
negative_train70 = [
    [335, 408, 50],[339, 405, 51],[340, 402, 52],[382, 440, 53],[244, 433, 53],[341, 402, 53],[391, 436, 54],[345, 431, 54],[343, 396, 54],[462, 438, 54],[243, 434, 54],[387, 433, 55],[352, 435, 55],[140, 473, 55],[116, 481, 55],[383, 427, 56],[460, 451, 56],[233, 405, 56],[134, 472, 56],[114, 479, 56],[466, 438, 57],[383, 425, 57],[385, 424, 57],[228, 405, 57],[200, 446, 57],[135, 470, 57],[110, 473, 57],[467, 437, 58],[382, 423, 58],[351, 432, 58],[185, 444, 58],[133, 468, 58],[109, 472, 58],[236, 401, 58],[467, 434, 59],[391, 423, 59],[332, 429, 59],[207, 435, 59],[116, 465, 59],[107, 470, 59],[231, 397, 59],[468, 432, 60],[392, 422, 60],[323, 437, 60],[187, 438, 60],[228, 395, 60],[119, 459, 60],[107, 469, 60],[469, 431, 61],[393, 423, 61],[184, 435, 61],[127, 458, 61],[115, 466, 61],[470, 429, 62],[391, 425, 62],[322, 394, 62],[202, 433, 62],[188, 435, 62],[112, 450, 62],[131, 467, 62],[100, 466, 62],[476, 447, 63],[464, 478, 63],[478, 436, 63],[397, 422, 63],[207, 434, 63],[184, 434, 63],[111, 443, 63],[115, 468, 63],[103, 466, 63],[477, 447, 64],[465, 437, 64],[400, 419, 64],[206, 435, 64],[184, 435, 64],[109, 438, 64],[130, 469, 64],[103, 467, 64],[476, 446, 65],[408, 416, 65],[398, 426, 65],[201, 435, 65],[183, 434, 65],[108, 435, 65],[127, 466, 65],[100, 464, 65],[481, 470, 66],[476, 446, 66],[476, 427, 66],[412, 414, 66],[400, 425, 66],[211, 436, 66],[183, 434, 66],[107, 433, 66],[130, 468, 66],[110, 461, 66],[476, 444, 67],[483, 461, 67],[411, 412, 67],[406, 423, 67],[199, 437, 67],[192, 434, 67],[106, 431, 67],[111, 465, 67],
    [139, 486, 67],[483, 460, 68],[478, 445, 68],[479, 424, 68],[409, 422, 68],[208, 437, 68],[190, 433, 68],[106, 431, 68],[110, 464, 68],[97, 459, 68],[98, 474, 68],[478, 422, 69],[478, 444, 69],[472, 461, 69],[418, 410, 69],[411, 421, 69],[194, 438, 69],[175, 433, 69],[104, 425, 69],[110, 456, 69],[109, 464, 69],[102, 478, 69],[121, 485, 69],[473, 460, 70],[467, 426, 70],[479, 414, 70],[412, 421, 70],[420, 409, 70],[192, 438, 70],[187, 432, 70],[104, 424, 70],[108, 462, 70],[108, 456, 70],[473, 458, 71],[476, 448, 71],[467, 425, 71],[480, 414, 71],[414, 420, 71],[421, 408, 71],[187, 439, 71],[171, 432, 71],[115, 423, 71],[108, 463, 71],[98, 455, 71],[452, 489, 71],[434, 490, 71],[452, 490, 72],[434, 490, 72],[423, 406, 72],[416, 418, 72],[183, 440, 72],[165, 432, 72],[114, 419, 72],[123, 460, 72],[93, 455, 72],[114, 480, 72],[475, 457, 73],[479, 442, 73],[424, 406, 73],[417, 418, 73],[182, 441, 73],[179, 430, 73],[114, 418, 73],[106, 452, 73],[106, 460, 73],[121, 475, 73],[480, 442, 74],[488, 454, 74],[417, 417, 74],[425, 406, 74],[185, 442, 74],[164, 430, 74],[114, 417, 74],[92, 452, 74],[104, 459, 74],[120, 475, 74],[476, 411, 75],[418, 416, 75],[160, 429, 75],[184, 442, 75],[102, 415, 75],[90, 452, 75],[104, 457, 75],[109, 473, 75],[476, 452, 76],[472, 442, 76],[162, 428, 76],[91, 450, 76],[103, 457, 76],[107, 471, 76],[476, 454, 77],[156, 427, 77],[183, 442, 77],[89, 450, 77],[104, 454, 77],[105, 469, 77],[476, 455, 78],[482, 442, 78],[154, 426, 78],[421, 414, 78],[183, 443, 78],[88, 449, 78],[97, 454, 78],[105, 468, 78],[476, 456, 79],
    [475, 454, 79],[483, 443, 79],[183, 443, 79],[161, 425, 79],[103, 448, 79],[89, 449, 79],[104, 466, 79],[481, 462, 80],[476, 453, 80],[484, 443, 80],[183, 443, 80],[158, 424, 80],[102, 447, 80],[103, 464, 80],[92, 449, 80],[484, 458, 81],[475, 454, 81],[484, 443, 81],[184, 444, 81],[151, 422, 81],[101, 446, 81],[88, 448, 81],[99, 462, 81],[470, 454, 82],[485, 459, 82],[485, 444, 82],[493, 483, 82],[180, 444, 82],[157, 423, 82],[100, 445, 82],[97, 461, 82],[88, 447, 82],[470, 453, 83],[478, 462, 83],[485, 445, 83],[418, 412, 83],[174, 445, 83],[159, 422, 83],[99, 444, 83],[88, 448, 83],[95, 458, 83],[496, 481, 84],[486, 464, 84],[475, 455, 84],[486, 445, 84],[421, 411, 84],[175, 446, 84],[160, 421, 84],[98, 442, 84],[87, 448, 84],[99, 456, 84],[101, 475, 84],[492, 478, 85],[487, 465, 85],[474, 454, 85],[486, 445, 85],[421, 411, 85],[185, 447, 85],[160, 421, 85],[98, 443, 85],[91, 448, 85],[95, 455, 85],[488, 467, 86],[474, 454, 86],[486, 446, 86],[474, 476, 86],[421, 410, 86],[178, 447, 86],[109, 438, 86],[90, 448, 86],[93, 454, 86],[489, 468, 87],[475, 475, 87],[474, 455, 87],[487, 446, 87],[492, 437, 87],[411, 410, 87],[188, 448, 87],[153, 419, 87],[96, 438, 87],[90, 448, 87],[94, 453, 87],[492, 471, 88],[473, 477, 88],[473, 456, 88],[487, 446, 88],[489, 440, 88],[371, 444, 88],[183, 450, 88],[158, 419, 88],[95, 438, 88],[84, 448, 88],[97, 451, 88],[101, 481, 88],[473, 455, 89],[498, 445, 89],[414, 410, 89],[180, 451, 89],[149, 418, 89],[95, 436, 89],[107, 447, 89],[84, 449, 89],[101, 482, 89],[473, 456, 90],[488, 441, 90],[416, 409, 90],
    [182, 452, 90],[150, 418, 90],[99, 442, 90],[107, 431, 90],[88, 449, 90],[472, 457, 91],[486, 445, 91],[412, 408, 91],[183, 454, 91],[154, 418, 91],[106, 428, 91],[95, 443, 91],[79, 449, 91],[101, 484, 91],[472, 458, 92],[486, 445, 92],[420, 409, 92],[188, 456, 92],[156, 417, 92],[91, 427, 92],[88, 447, 92],[94, 440, 92],[100, 485, 92],[472, 460, 93],[484, 444, 93],[419, 409, 93],[388, 411, 93],[150, 417, 93],[150, 417, 93],[101, 440, 93],[90, 443, 93],[87, 449, 93],[99, 486, 93],[193, 456, 93],[470, 456, 94],[489, 446, 94],[419, 409, 94],[149, 416, 94],[100, 437, 94],[88, 444, 94],[85, 450, 94],[198, 457, 94],[98, 486, 94],[90, 422, 94],[471, 466, 95],[469, 457, 95],[489, 447, 95],[484, 443, 95],[385, 411, 95],[419, 409, 95],[152, 417, 95],[192, 458, 95],[97, 486, 95],[95, 496, 95],[86, 449, 95],[91, 442, 95],[100, 437, 95],[469, 457, 96],[470, 458, 96],[489, 447, 96],[475, 442, 96],[418, 409, 96],[153, 416, 96],[199, 458, 96],[86, 451, 96],[91, 442, 96],[94, 436, 96],[95, 485, 96],[469, 467, 97],[465, 455, 97],[490, 447, 97],[414, 408, 97],[200, 459, 97],[154, 416, 97],[102, 432, 97],[86, 441, 97],[85, 449, 97],[93, 486, 97],[462, 471, 98],[467, 454, 98],[489, 447, 98],[476, 434, 98],[408, 408, 98],[200, 460, 98],[154, 416, 98],[102, 431, 98],[82, 439, 98],[84, 450, 98],[93, 485, 98],[464, 471, 99],[461, 453, 99],[417, 408, 99],[201, 461, 99],[154, 415, 99],[103, 425, 99],[88, 439, 99],[84, 450, 99],[464, 472, 100],[465, 454, 100],[475, 432, 100],[490, 448, 100],[417, 409, 100],[161, 414, 100],[364, 455, 100],[80, 438, 100],[84, 451, 100],
]     
semble = negative_train70
for n in range(len(semble)):
    x,y,z = semble[n]
    semble[n] = [int(round((x+0.5)*insert0-1)),int(round((y+0.5)*insert1-1)),int(round((z+0.5)*insert2-1))]
a1,a2,b1,b2,c1,c2 = 404,455,437,462,61,65
a1 = int(a1*insert0)
a2 = int(a2*insert0)
b1 = int(b1*insert1)
b2 = int(b2*insert1)
c1 = int(c1*insert2)
c2 = int(c2*insert2)               
for x in range(a1,a2,4):
    for y in range(b1,b2,4):
        for z in range(c1,c2,4):
            if target70_3d_normalization[x,y,z]<30:
                negative_train70.append([x,y,z])

In [ ]:
#S65TRN
negative_train65 = []                
for x in range(int(370*insert0),int(400*insert0),3):
    for y in range(int(430*insert1),int(470*insert1),3):
        for z in range(int(17*insert2),int(23*insert2),3):
            if target65_3d_normalization[x,y,z]<30:
                negative_train65.append([x,y,z])

# y_train_0 = np.zeros((len(negative_train)+len(negative_train65)+len(negative_train67)+len(negative_train70)+len(negative_train77)))
# y_train_before = np.hstack((y_train_1,y_train_0))

In [ ]:
#微血管训练集-S57p
complex_train = []
label_train = []
a,b,c = target_3d_normalization.shape
for n in range(len(positive_train)):
    x,y,z = positive_train[n]
    x=int(round((x+0.5)*insert0-1))
    y=int(round((y+0.5)*insert1-1))
    z=int(round((z+0.5)*insert2-1))
    
    if target_3d_normalization[x,y,z] > 1:
        region = np.zeros((patch0,patch1,patch2))         
        pre = []
        for i in range(patch0):
            for j in range(patch1):
                for k in range(patch2):
                    region[i,j,k] = target_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
                    pre.append(target_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)])
        past = 255/max(pre)
        region = np.array(region)
        region_past = region*past
        complex_train.append(region)
#         region_small = region[int((patch0-pdt_patch0)/2):int((patch0-pdt_patch0)/2)+pdt_patch0,int((patch1-pdt_patch1)/2):int((patch1-pdt_patch1)/2)+pdt_patch1,int((patch2-pdt_patch2)/2):int((patch2-pdt_patch2)/2)+pdt_patch2]
#         label_region = np.zeros((pdt_patch0,pdt_patch1,pdt_patch2))
#         for i in range(pdt_patch0):
#             for j in range(pdt_patch1):
#                 for k in range(pdt_patch2):
#                     if region_small[i,j,k]>region_small[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]*0.7 or region_small[i,j,k]>35:
#                         label_region[i,j,k]=1
#         label_region[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]=1
#         label_train.append(label_region)
#微血管训练集-S65p
a,b,c = target65_3d_normalization.shape
for n in range(len(positive_train65)):
    x,y,z = positive_train65[n]
    x=int(round((x+0.5)*insert0-1))
    y=int(round((y+0.5)*insert1-1))
    z=int(round((z+0.5)*insert2-1))
    pre = []
    if target65_3d_normalization[x,y,z] > 1:
        region = np.zeros((patch0,patch1,patch2))
        for i in range(patch0):
            for j in range(patch1):
                for k in range(patch2):
                    region[i,j,k] = target65_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
                    pre.append(target65_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)])
        past = 255/max(pre)
        region = np.array(region)
        region_past = region*past
        complex_train.append(region)
#         region_small = region[int((patch0-pdt_patch0)/2):int((patch0-pdt_patch0)/2)+pdt_patch0,int((patch1-pdt_patch1)/2):int((patch1-pdt_patch1)/2)+pdt_patch1,int((patch2-pdt_patch2)/2):int((patch2-pdt_patch2)/2)+pdt_patch2]
#         label_region = np.zeros((pdt_patch0,pdt_patch1,pdt_patch2))
#         for i in range(pdt_patch0):
#             for j in range(pdt_patch1):
#                 for k in range(pdt_patch2):
#                     if region_small[i,j,k]>region_small[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]*0.7 or region_small[i,j,k]>35:
#                         label_region[i,j,k]=1
#         label_region[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]=1
#         label_train.append(label_region)
    
#微血管训练集-S67p
a,b,c = target67_3d_normalization.shape
for n in range(len(positive_train67)):
    x,y,z = positive_train67[n]
    x=int(round((x+0.5)*insert0-1))
    y=int(round((y+0.5)*insert1-1))
    z=int(round((z+0.5)*insert2-1))
    pre = []
    if target67_3d_normalization[x,y,z] > 1:
        region = np.zeros((patch0,patch1,patch2))
        for i in range(patch0):
            for j in range(patch1):
                for k in range(patch2):
                    region[i,j,k] = target67_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
                    pre.append(target67_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)])
        past = 255/max(pre)
        region = np.array(region)
        region_past = region*past
        complex_train.append(region)
#         region_small = region[int((patch0-pdt_patch0)/2):int((patch0-pdt_patch0)/2)+pdt_patch0,int((patch1-pdt_patch1)/2):int((patch1-pdt_patch1)/2)+pdt_patch1,int((patch2-pdt_patch2)/2):int((patch2-pdt_patch2)/2)+pdt_patch2]
#         label_region = np.zeros((pdt_patch0,pdt_patch1,pdt_patch2))
#         for i in range(pdt_patch0):
#             for j in range(pdt_patch1):
#                 for k in range(pdt_patch2):
#                     if region_small[i,j,k]>region_small[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]*0.7 or region_small[i,j,k]>35:
#                         label_region[i,j,k]=1
#         label_region[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]=1
#         label_train.append(label_region)
    
#微血管训练集-S70p
a,b,c = target70_3d_normalization.shape
for n in range(len(positive_train70)):
    x,y,z = positive_train70[n]
    x=int(round((x+0.5)*insert0-1))
    y=int(round((y+0.5)*insert1-1))
    z=int(round((z+0.5)*insert2-1))
    pre = []
    if target70_3d_normalization[x,y,z] > 1:
        region = np.zeros((patch0,patch1,patch2))
        for i in range(patch0):
            for j in range(patch1):
                for k in range(patch2):
                    region[i,j,k] = target70_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
                    pre.append(target70_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)])
        past = 255/max(pre)
        region = np.array(region)
        region_past = region*past
        complex_train.append(region)
#         region_small = region[int((patch0-pdt_patch0)/2):int((patch0-pdt_patch0)/2)+pdt_patch0,int((patch1-pdt_patch1)/2):int((patch1-pdt_patch1)/2)+pdt_patch1,int((patch2-pdt_patch2)/2):int((patch2-pdt_patch2)/2)+pdt_patch2]
#         label_region = np.zeros((pdt_patch0,pdt_patch1,pdt_patch2))
#         for i in range(pdt_patch0):
#             for j in range(pdt_patch1):
#                 for k in range(pdt_patch2):
#                     if region_small[i,j,k]>region_small[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]*0.7 or region_small[i,j,k]>35:
#                         label_region[i,j,k]=1
#         label_region[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]=1
#         label_train.append(label_region)
    
#微血管训练集-S77p
a,b,c = target77_3d_normalization.shape
for n in range(len(positive_train77)):
    x,y,z = positive_train77[n]
    x=int(round((x+0.5)*insert0-1))
    y=int(round((y+0.5)*insert1-1))
    z=int(round((z+0.5)*insert2-1))
    pre = []
    if target77_3d_normalization[x,y,z] > 1:
        region = np.zeros((patch0,patch1,patch2))
        for i in range(patch0):
            for j in range(patch1):
                for k in range(patch2):
                    region[i,j,k] = target77_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
                    pre.append(target77_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)])
        past = 255/max(pre)
        region = np.array(region)
        region_past = region*past
        complex_train.append(region)
#         region_small = region[int((patch0-pdt_patch0)/2):int((patch0-pdt_patch0)/2)+pdt_patch0,int((patch1-pdt_patch1)/2):int((patch1-pdt_patch1)/2)+pdt_patch1,int((patch2-pdt_patch2)/2):int((patch2-pdt_patch2)/2)+pdt_patch2]
#         label_region = np.zeros((pdt_patch0,pdt_patch1,pdt_patch2))
#         for i in range(pdt_patch0):
#             for j in range(pdt_patch1):
#                 for k in range(pdt_patch2):
#                     if region_small[i,j,k]>region_small[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]*0.7 or region_small[i,j,k]>35:
#                         label_region[i,j,k]=1
#         label_region[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]=1
#         label_train.append(label_region)

In [ ]:
y_train_1 = np.ones(len(complex_train))
half_train = len(complex_train)

In [ ]:
#微血管训练集-S57n
a,b,c = target_3d_normalization.shape
for n in range(len(negative_train)):
    x,y,z = negative_train[n]
#     x=int(round(x*insert0))
#     y=int(round(y*insert1))
#     z=int(round(z*insert2))
    pre = []
    if target_3d_normalization[x,y,z] < 100:
        region = np.zeros((patch0,patch1,patch2))
        for i in range(patch0):
            for j in range(patch1):
                for k in range(patch2):
                    region[i,j,k] = target_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
                    pre.append(target_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)])
        past = 255/max(pre)
        region = np.array(region)
        region_past = region*past
        complex_train.append(region)
#         region_small = region[int((patch0-pdt_patch0)/2):int((patch0-pdt_patch0)/2)+pdt_patch0,int((patch1-pdt_patch1)/2):int((patch1-pdt_patch1)/2)+pdt_patch1,int((patch2-pdt_patch2)/2):int((patch2-pdt_patch2)/2)+pdt_patch2]
#         label_region = np.zeros((pdt_patch0,pdt_patch1,pdt_patch2))
#         for i in range(pdt_patch0):
#             for j in range(pdt_patch1):
#                 for k in range(pdt_patch2):
#                     if region_small[i,j,k]>region_small[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]*1.2 and region_small[i,j,k]>40:
#                         label_region[i,j,k]=1
#         label_region[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]=0
#         label_train.append(label_region)

#S65N
for n in range(len(negative_train65)):
    x,y,z = negative_train65[n]
#     x=int(round(x*insert0))
#     y=int(round(y*insert1))
#     z=int(round(z*insert2))
    pre = []
    if target65_3d_normalization[x,y,z] < 100:
        region = np.zeros((patch0,patch1,patch2))
        for i in range(patch0):
            for j in range(patch1):
                for k in range(patch2):
                    region[i,j,k] = target65_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
                    pre.append(target65_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)])
        past = 255/max(pre)
        region = np.array(region)
        region_past = region*past
        complex_train.append(region)
#         region_small = region[int((patch0-pdt_patch0)/2):int((patch0-pdt_patch0)/2)+pdt_patch0,int((patch1-pdt_patch1)/2):int((patch1-pdt_patch1)/2)+pdt_patch1,int((patch2-pdt_patch2)/2):int((patch2-pdt_patch2)/2)+pdt_patch2]
#         label_region = np.zeros((pdt_patch0,pdt_patch1,pdt_patch2))
#         for i in range(pdt_patch0):
#             for j in range(pdt_patch1):
#                 for k in range(pdt_patch2):
#                     if region_small[i,j,k]>region_small[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]*1.2 and region_small[i,j,k]>40:
#                         label_region[i,j,k]=1
#         label_region[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]=0
#         label_train.append(label_region)

    
#微血管训练集-S67n
a,b,c = target67_3d_normalization.shape
for n in range(len(negative_train67)):
    x,y,z = negative_train67[n]
#     x=int(round(x*insert0))
#     y=int(round(y*insert1))
#     z=int(round(z*insert2))
    if target67_3d_normalization[x,y,z] < 300:
        region = np.zeros((patch0,patch1,patch2))
        for i in range(patch0):
            for j in range(patch1):
                for k in range(patch2):
                    region[i,j,k] = target67_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
        complex_train.append(region)
    
#微血管训练集-S70n
a,b,c = target70_3d_normalization.shape
for n in range(len(negative_train70)):
    x,y,z = negative_train70[n]
#     x=int(round(x*insert0))
#     y=int(round(y*insert1))
#     z=int(round(z*insert2))
    if target70_3d_normalization[x,y,z] < 300:
        region = np.zeros((patch0,patch1,patch2))
        for i in range(patch0):
            for j in range(patch1):
                for k in range(patch2):
                    region[i,j,k] = target70_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
        complex_train.append(region)
    
#微血管训练集-S77n
a,b,c = target77_3d_normalization.shape
for n in range(len(negative_train77)):
    x,y,z = negative_train77[n]
    if target77_3d_normalization[x,y,z] < 300:
        region = np.zeros((patch0,patch1,patch2))
        for i in range(patch0):
            for j in range(patch1):
                for k in range(patch2):
                    region[i,j,k] = target77_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
        complex_train.append(region)

In [ ]:
y_train_0 = np.zeros(len(complex_train)-half_train)
y_train_before = np.hstack((y_train_1,y_train_0))

In [ ]:
positive_test = [
    [332, 445, 25], [331, 445, 25], [330, 445, 25], [333, 444, 25], [332, 444, 25], [331, 444, 25], [330, 444, 25], [332, 443, 25], 
    [331, 443, 25], [330, 443, 25], [332, 442, 25], [331, 442, 25], [330, 442, 25], [331, 441, 25], [330, 441, 25], [329, 441, 25], 
    [331, 440, 25], [330, 440, 25], [329, 440, 25], [331, 439, 25], [330, 439, 25], [329, 439, 25], [328, 439, 25], [331, 438, 25], 
    [330, 438, 25], [329, 438, 25], [330, 437, 25], [329, 437, 25], [330, 436, 25], [329, 436, 25], [328, 436, 25], [330, 435, 25],
    [330, 427, 28],[329, 427, 28],[331, 426, 28],[330, 426, 28],[329, 426, 28],[332, 425, 28],[331, 425, 28],[330, 425, 28],[332, 424, 28],[331, 424, 28],
    [330, 424, 28],[332, 423, 28],[331, 423, 28],[333, 448, 29],[333, 447, 29],[332, 447, 29],[333, 446, 29],[332, 446, 29],[339, 446, 29],[338, 446, 29],
    [329, 435, 25], [329, 434, 25], [328, 433, 26],[363, 458, 30],[368, 458, 30],[363, 459, 30]               
]

# for x in range(180,320):
#     for y in range(390,500):
#         for z in range(30,40):
#             if target_3d_normalization[x,y,z]>90:
#                 positive_test.append([x,y,z])

positive_test65 = [
    [369, 456, 72],[362, 457, 73],[363, 457, 73],[364, 457, 73],[365, 457, 73],[366, 458, 73],[366, 457, 73],[367, 458, 73],[367, 457, 73],
    [367, 456, 73],[367, 455, 73],[368, 459, 73],[368, 458, 73],[368, 457, 73],[368, 456, 73],[369, 459, 73],[369, 458, 73],[369, 457, 73],
    [362, 457, 74],[362, 456, 74],[363, 457, 74],[363, 456, 74],[364, 457, 74],[365, 457, 74],[364, 452, 74],[364, 451, 74],[364, 450, 74],
    [364, 452, 73],[364, 451, 73],[364, 450, 73],[364, 452, 67],[364, 451, 67]
]

positive_test67 = [
    [61, 495, 81],[60, 495, 81],[67, 496, 81],[66, 496, 81],[65, 496, 81],[64, 496, 81],[64, 496, 81],[63, 496, 81],[62, 496, 81],
    [61, 496, 81],[463, 438, 88],[463, 437, 88],[461, 438, 88],[463, 439, 87],[463, 438, 87],[463, 437, 87],[463, 436, 87],[463, 435, 87],
    [463, 434, 87],[463, 433, 87],[463, 432, 87],[464, 440, 87],[464, 439, 87],[464, 438, 87],[464, 437, 87],[464, 436, 87],[464, 435, 87],
    [464, 434, 87],[464, 433, 87],[464, 432, 87] 
]
# for x in range(90,430):
#     for y in range(240,520):
#         for z in range(40,45):
#             if target67_3d_normalization[x,y,z] > 80:
#                 positive_test67.append([x,y,z])

positive_test70 = [
    [175, 410, 113],[465, 444, 114],[462, 415, 114],[414, 409, 114],[325, 402, 114],[108, 402, 114],[466, 444, 115],[461, 414, 115],
    [413, 408, 115],[325, 402, 115],[466, 443, 116],[460, 415, 116],[327, 401, 116],[272, 409, 116],[178, 407, 116],[466, 440, 117],
    [459, 415, 117],[411, 405, 117],[328, 401, 117],[271, 410, 117],[179, 407, 117],[109, 400, 117],[470, 441, 118],[458, 414, 118],
    [330, 400, 118],[267, 412, 118],[99, 435, 118],[472, 441, 119],[457, 412, 119],[330, 400, 119],[266, 412, 119],[112, 437, 119],
    [110, 398, 120],[264, 411, 120],[332, 397, 120],[456, 412, 120],[115, 436, 121],[281, 397, 121],[263, 410, 121],[333, 396, 121],
    [404, 402, 121],[456, 412, 121],[263, 410, 122],[333, 396, 122],[455, 410, 122]
]

# a1,a2,b1,b2,c1,c2 = 150,240,270,400,40,45
# a1 = int(a1*insert0)
# a2 = int(a2*insert0)
# b1 = int(b1*insert1)
# b2 = int(b2*insert1)
# c1 = int(c1*insert2)
# c2 = int(c2*insert2)               
# for x in range(a1,a2):
#     for y in range(b1,b2):
#         for z in range(c1,c2):
#             if target70_3d_normalization[x,y,z] > 80:
#                 positive_test70.append([x,y,z])

positive_test77 = [
    [132, 404, 96],[129, 391, 96],[138, 356, 96],[458, 390, 97],[465, 377, 97],[466, 358, 97],[457, 363, 97],[105, 420, 97],[133, 402, 97],
    [129, 390, 97],[139, 356, 97],[457, 387, 98],[463, 378, 98],[458, 358, 98],[105, 421, 98],[135, 400, 98],[129, 389, 98],[140, 357, 98],
    [463, 377, 99],[459, 359, 99],[464, 354, 99],[103, 420, 99],[135, 397, 99],[129, 388, 99],[141, 358, 99],[461, 379, 100],[457, 359, 100],
    [465, 347, 100],[101, 418, 100],[133, 396, 100],[126, 387, 100],[142, 359, 100],[460, 380, 101],[460, 357, 101],[466, 346, 101],
    [101, 419, 101],[135, 395, 101],[129, 387, 101],[142, 360, 101],[460, 355, 102],[466, 347, 102],[100, 418, 102],[135, 393, 102],
    [129, 385, 102],[143, 362, 102],[460, 354, 103],[464, 342, 103],[96, 416, 103],[135, 392, 103],[129, 383, 103],[144, 364, 103],
    [458, 354, 104],[463, 343, 104],[144, 365, 104],[94, 415, 104],[135, 391, 104],[130, 384, 104],[462, 351, 105],[465, 342, 105],
    [144, 366, 105],[94, 415, 105],[135, 389, 105],[129, 381, 105],[462, 350, 106],[144, 366, 106],[94, 416, 106],[134, 388, 106],
    [130, 381, 106],[463, 349, 107],[143, 367, 107],[91, 415, 107],[134, 389, 107],[130, 380, 107],[463, 345, 108],[143, 367, 108],
    [88, 413, 108],[132, 387, 108],[130, 378, 108],[463, 343, 109],[142, 367, 109],[86, 411, 109],[133, 389, 109],[131, 378, 109],
    [465, 348, 110],[462, 337, 110],[141, 368, 110],[131, 377, 110],[131, 387, 110],[469, 346, 111],[462, 339, 111],[141, 368, 111],
    [129, 378, 111],[131, 389, 111],[472, 346, 112],[461, 337, 112],[129, 389, 112],[132, 375, 112],[140, 368, 112],[473, 346, 113],
    [458, 336, 113],[132, 375, 113],[128, 390, 113],[140, 368, 113],[460, 337, 114],[131, 375, 114],[126, 391, 114],[459, 338, 115],
    [132, 373, 115],[124, 386, 115],[456, 335, 116],[134, 371, 116],[124, 386, 116],[458, 335, 117],[135, 370, 117],[130, 380, 117],
    [457, 334, 118],[136, 370, 118],[131, 380, 118],[457, 334, 119],[137, 367, 119],[133, 379, 119],[453, 332, 120],[138, 366, 120],
    [134, 380, 120],[454, 331, 121],[455, 331, 121],[456, 331, 121],[449, 331, 121],[139, 365, 121],[142, 365, 121],[137, 365, 121],
    [134, 380, 121],[135, 380, 121],[453, 330, 122],[455, 330, 122],[448, 330, 122],[139, 363, 122],[143, 363, 122],[138, 363, 122],
    [135, 380, 122],[134, 380, 122]
]
for x in range(110,400):
    for y in range(250,530):
        for z in range(60,65):
            if target77_3d_normalization[x,y,z] > 80:
                positive_test77.append([x,y,z])

y_test_1 = np.ones((len(positive_test)+len(positive_test65)+len(positive_test67)+len(positive_test70)+len(positive_test77)))

In [ ]:
negative_test = [
[96,535,58],[176,540,58],[204,531,58],[400,468,59],[405,462,59],[101,451,59],[154,443,59],[235,418,55],[408,418,55],[422,446,55],
[398,462,46],[171,446,46],[91,460,46],[282,396,46],[181,342,43],[204,336,43],[278,276,43],[286,276,43],[295,275,43],[350,293,43],
[282,289,34],[280,280,34],[236,285,34],[163,286,34],[190,345,34],[180,356,34],[393,334,34],[212,412,34],[284,418,34],[352,409,34],
[282,389,38],[228,409,38],[396,621,38],[456,530,38],[445,514,38],[456,503,38],[385,455,38],[201,460,38],[279,523,38],[221,254,28],
[461,422,38],[422,358,38],[396,322,38],[354,314,38],[398,267,28],[380,225,28],[391,200,28],[350,200,28],[212,232,26],[232,179,26],
[341, 427, 28],[339, 427, 28],[326, 426, 28],[325, 426, 28],[340, 426, 28],[326, 425, 28],[325, 425, 28],[341, 425, 28],[330, 424, 28],[325, 424, 28],
[341, 424, 28],[340, 423, 28],[341, 423, 28],[324, 448, 29],[326, 447, 29],[324, 447, 29],[337, 446, 29],[329, 446, 29],[328, 446, 29],[336, 446, 29],
[325, 434, 25],[341, 451, 26],[342, 451, 26],[335, 451, 26],[334, 451, 26],[330, 451, 26],[327, 451, 26],[329, 450, 26],[342, 449, 26],[331, 449, 26],
[333, 449, 26],[330, 449, 26],[336, 448, 26],[329, 448, 26],[342, 448, 26],[339, 447, 26],[336, 447, 26],[328, 447, 26],[326, 447, 26],[344, 447, 26],
[340, 446, 26],[339, 446, 26],[329, 446, 26],[328, 446, 26],[327, 446, 26],[330, 444, 26],[326, 444, 26],[327, 444, 26],[326, 443, 26],[335, 443, 26],
[329, 442, 26],[325, 441, 26],[328, 441, 26],[335, 440, 26],[325, 440, 26],[325, 439, 26],[329, 439, 26],[324, 438, 26],[335, 438, 26],[323, 438, 26],
[327, 437, 26],[323, 437, 26],[324, 436, 26],[335, 436, 26],[328, 435, 26],[323, 435, 26],[328, 434, 26],[325, 434, 26],[324, 433, 26],[334, 433, 26],
[365, 458, 30],[361, 458, 30],[360, 459, 30]
]
semble = negative_test
for n in range(len(semble)):
    x,y,z = semble[n]
    semble[n] = [int(round(x*insert0)),int(round(y*insert1)),int(round(z*insert2))]
a1,a2,b1,b2,c1,c2 = 180,220,490,500,34,37
a1 = int(a1*insert0)
a2 = int(a2*insert0)
b1 = int(b1*insert1)
b2 = int(b2*insert1)
c1 = int(c1*insert2)
c2 = int(c2*insert2)               
for x in range(a1,a2,6):
    for y in range(b1,b2,6):
        for z in range(c1,c2,6):
            if target_3d_normalization[x,y,z]<30:
                negative_test.append([x,y,z])

a1,a2,b1,b2,c1,c2 = 363,448,405,433,32,34
a1 = int(a1*insert0)
a2 = int(a2*insert0)
b1 = int(b1*insert1)
b2 = int(b2*insert1)
c1 = int(c1*insert2)
c2 = int(c2*insert2)               
for x in range(a1,a2,6):
    for y in range(b1,b2,6):
        for z in range(c1,c2,6):
            if target_3d_normalization[x,y,z]<30:
                negative_test.append([x,y,z])

negative_test65 = []
a1,a2,b1,b2,c1,c2 = 370,400,430,470,23,26
a1 = int(a1*insert0)
a2 = int(a2*insert0)
b1 = int(b1*insert1)
b2 = int(b2*insert1)
c1 = int(c1*insert2)
c2 = int(c2*insert2)               
for x in range(a1,a2,6):
    for y in range(b1,b2,6):
        for z in range(c1,c2,6):
            if target65_3d_normalization[x,y,z]<30:
                negative_test65.append([x,y,z])
                
negative_test67 = [
    [463, 431, 88],[463, 427, 88],[463, 450, 88],[460, 450, 88],[452, 430, 77],[451, 430, 77],[455, 426, 77],[455, 409, 88],[455, 391, 88],
    [455, 390, 88],[455, 389, 88],[455, 410, 90],[455, 409, 90],[420, 425, 77],[462, 460, 79],[462, 459, 79],[462, 458, 79],[462, 453, 79],
    [462, 457, 80],[462, 454, 80]
]
semble = negative_test67
for n in range(len(semble)):
    x,y,z = semble[n]
    semble[n] = [int(round(x*insert0)),int(round(y*insert1)),int(round(z*insert2))]
a1,a2,b1,b2,c1,c2 = 330,415,420,475,65,67
a1 = int(a1*insert0)
a2 = int(a2*insert0)
b1 = int(b1*insert1)
b2 = int(b2*insert1)
c1 = int(c1*insert2)
c2 = int(c2*insert2)               
for x in range(a1,a2,5):
    for y in range(b1,b2,5):
        for z in range(c1,c2,5):
            if target67_3d_normalization[x,y,z]<30:
                negative_test67.append([x,y,z])

negative_test70 = [
    [457, 472, 101],[462, 453, 101],[405, 409, 101],[202, 464, 101],[152, 415, 101],[83, 452, 101],[87, 441, 101],[476, 472, 102],
    [462, 455, 102],[416, 409, 102],[468, 425, 102],[162, 415, 102],[474, 473, 103],[462, 453, 103],[467, 424, 103],[415, 409, 103],
    [160, 414, 103],[104, 414, 103],[460, 423, 104],[462, 452, 104],[474, 472, 104],[407, 407, 104],[160, 414, 104],[460, 422, 105],
    [416, 409, 105],[461, 451, 105],[166, 414, 105],[103, 411, 105],[460, 449, 106],[416, 409, 106],[157, 414, 106],[459, 450, 107],
    [415, 408, 107],[157, 414, 107],[454, 473, 108],[453, 448, 108],[462, 418, 108],[410, 408, 108],[160, 413, 108],[456, 476, 109],
    [456, 449, 109],[461, 417, 109],[414, 409, 109],[166, 413, 109],[459, 446, 110],[455, 476, 110],[455, 447, 110],[457, 416, 110],
    [412, 409, 110],[169, 412, 110],[102, 406, 110],[449, 474, 111],[458, 445, 111],[461, 416, 111],[413, 409, 111],[170, 411, 111],
    [470, 446, 112],[460, 415, 112],[412, 409, 112],[166, 411, 112],[459, 444, 113],[449, 414, 113],[411, 409, 113],[315, 403, 113],
    [171, 410, 113],[453, 444, 114],[458, 415, 114],[403, 409, 114],[316, 402, 114],[103, 402, 114],[458, 444, 115],[457, 414, 115],
    [402, 408, 115],[321, 402, 115],[459, 443, 116],[462, 415, 116],[318, 401, 116],[263, 409, 116],[170, 407, 116],[460, 440, 117],
    [461, 415, 117],[404, 405, 117],[319, 401, 117],[277, 410, 117],[175, 407, 117],[102, 400, 117],[461, 441, 118],[449, 414, 118],
    [322, 400, 118],[257, 412, 118],[91, 435, 118],[462, 441, 119],[447, 412, 119],[326, 400, 119],[256, 412, 119],[109, 437, 119],
    [102, 398, 120],[251, 411, 120]
]
semble = negative_test70
for n in range(len(semble)):
    x,y,z = semble[n]
    semble[n] = [int(round(x*insert0)),int(round(y*insert1)),int(round(z*insert2))]
a1,a2,b1,b2,c1,c2 = 404,455,437,462,65,67
a1 = int(a1*insert0)
a2 = int(a2*insert0)
b1 = int(b1*insert1)
b2 = int(b2*insert1)
c1 = int(c1*insert2)
c2 = int(c2*insert2)               
for x in range(a1,a2,3):
    for y in range(b1,b2,3):
        for z in range(c1,c2,3):
            if target70_3d_normalization[x,y,z]<30:
                negative_test70.append([x,y,z])
[322, 397, 120],[453, 412, 120],[120, 436, 121],[276, 397, 121],[253, 410, 121],[326, 396, 121],[398, 402, 121],[453, 412, 121],[253, 410, 122],
    [327, 396, 122],[450, 410, 122],[114, 436, 123],[106, 396, 123],[257, 410, 123],[328, 391, 123],[106, 396, 124],[102, 433, 124],[248, 407, 124],
    [327, 391, 124],[450, 410, 124],[454, 437, 124],[104, 433, 125],[255, 406, 125],[323, 390, 125],[392, 416, 125],[449, 409, 125],[458, 437, 125],
    [106, 432, 126],[255, 405, 126],[329, 387, 126],[444, 407, 126],[119, 436, 127],[254, 405, 127],[323, 385, 127],[451, 407, 127]

negative_test77 = [
    [466, 357, 101],[470, 346, 101],[106, 419, 101],[130, 395, 101],[132, 387, 101],[140, 360, 101],[463, 355, 102],[469, 347, 102],
    [105, 418, 102],[128, 393, 102],[121, 385, 102],[137, 362, 102],[464, 354, 103],[469, 342, 103],[103, 416, 103],[130, 392, 103],
    [132, 383, 103],[138, 364, 103],[464, 354, 104],[458, 343, 104],[148, 365, 104],[104, 415, 104],[140, 391, 104],[133, 384, 104],
    [465, 351, 105],[467, 342, 105],[148, 366, 105],[100, 415, 105],[125, 389, 105],[117, 381, 105],[466, 350, 106],[147, 366, 106],
    [99, 416, 106],[139, 388, 106],[133, 381, 106],[468, 349, 107],[145, 367, 107],[99, 415, 107],[126, 389, 107],[134, 380, 107],
    [460, 345, 108],[146, 367, 108],[98, 413, 108],[136, 387, 108],[124, 378, 108],[467, 343, 109],[136, 367, 109],[92, 411, 109],
    [126, 389, 109],[135, 378, 109],[472, 348, 110],[464, 337, 110],[145, 368, 110],[136, 377, 110],[135, 387, 110],[461, 346, 111],
    [453, 339, 111],[134, 368, 111],[136, 378, 111],[135, 389, 111],[462, 346, 112],[454, 337, 112],[134, 389, 112],[126, 375, 112],
    [135, 368, 112],[483, 346, 113],[453, 336, 113],[138, 375, 113],[131, 390, 113],[143, 368, 113],[463, 337, 114],[138, 375, 114],
    [129, 391, 114],[462, 338, 115],[128, 373, 115],[130, 386, 115],[461, 335, 116],[141, 371, 116],[121, 386, 116],[461, 335, 117],
    [141, 370, 117],[126, 380, 117],[460, 334, 118],[142, 370, 118],[127, 380, 118],[460, 334, 119],[145, 367, 119],[130, 379, 119],
    [459, 332, 120],[144, 366, 120],[137, 380, 120],[458, 331, 121],[144, 365, 121],[137, 380, 121],[444, 330, 122],[145, 363, 122],
    [132, 380, 122]
]

for x in range(350,390):
    for y in range(430,430):
        for z in range(65,67):
            if target77_3d_normalization[x,y,z]<30:
                negative_test77.append([x,y,z])

y_test_0 = np.zeros((len(negative_test)+len(negative_test65)+len(negative_test67)+len(negative_test70)+len(negative_test77)))
y_test_before = np.hstack((y_test_1,y_test_0))

In [ ]:
#微血管测试集-S57p
complex_test = []
label_test = []
label_valid = []
a,b,c = target_3d_normalization.shape
for n in range(len(positive_test)):
    x,y,z = positive_test[n]
    x=int(round((x+0.5)*insert0-1))
    y=int(round((y+0.5)*insert1-1))
    z=int(round((z+0.5)*insert2-1))
    pre = []
    if target_3d_normalization[x,y,z] > 10:
        region = np.zeros((patch0,patch1,patch2))
        for i in range(patch0):
            for j in range(patch1):
                for k in range(patch2):
                    region[i,j,k] = target_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
                    pre.append(target_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)])
        past = 255/max(pre)
        region = np.array(region)
        region_past = region*past
        complex_test.append(region)
#         region_small = region[int((patch0-pdt_patch0)/2):int((patch0-pdt_patch0)/2)+pdt_patch0,int((patch1-pdt_patch1)/2):int((patch1-pdt_patch1)/2)+pdt_patch1,int((patch2-pdt_patch2)/2):int((patch2-pdt_patch2)/2)+pdt_patch2]
#         label_region = np.zeros((pdt_patch0,pdt_patch1,pdt_patch2))
#         for i in range(pdt_patch0):
#             for j in range(pdt_patch1):
#                 for k in range(pdt_patch2):
#                     if region_small[i,j,k]>region_small[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]*0.7 or region_small[i,j,k]>35:
#                         label_region[i,j,k]=1
#         label_region[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]=1
#         label_test.append(label_region)
#         label_valid.append(region_small)

#微血管测试集-S65p
a,b,c = target65_3d_normalization.shape
for n in range(len(positive_test65)):
    x,y,z = positive_test65[n]
    x=int(round((x+0.5)*insert0-1))
    y=int(round((y+0.5)*insert1-1))
    z=int(round((z+0.5)*insert2-1))
    pre = []
    if target65_3d_normalization[x,y,z] > 10:
        region = np.zeros((patch0,patch1,patch2))
        for i in range(patch0):
            for j in range(patch1):
                for k in range(patch2):
                    region[i,j,k] = target65_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
                    pre.append(target65_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)])
        past = 255/max(pre)
        region = np.array(region)
        region_past = region*past
        complex_test.append(region)
#         region_small = region[int((patch0-pdt_patch0)/2):int((patch0-pdt_patch0)/2)+pdt_patch0,int((patch1-pdt_patch1)/2):int((patch1-pdt_patch1)/2)+pdt_patch1,int((patch2-pdt_patch2)/2):int((patch2-pdt_patch2)/2)+pdt_patch2]
#         label_region = np.zeros((pdt_patch0,pdt_patch1,pdt_patch2))
#         for i in range(pdt_patch0):
#             for j in range(pdt_patch1):
#                 for k in range(pdt_patch2):
#                     if region_small[i,j,k]>region_small[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]*0.7 or region_small[i,j,k]>35:
#                         label_region[i,j,k]=1
#         label_region[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]=1
#         label_test.append(label_region)
#         label_valid.append(region_small)
    
#微血管测试集-S67p
a,b,c = target67_3d_normalization.shape
for n in range(len(positive_test67)):
    x,y,z = positive_test67[n]
    x=int(round((x+0.5)*insert0-1))
    y=int(round((y+0.5)*insert1-1))
    z=int(round((z+0.5)*insert2-1))
    pre = []
    if target67_3d_normalization[x,y,z] > 10:
        region = np.zeros((patch0,patch1,patch2))
        for i in range(patch0):
            for j in range(patch1):
                for k in range(patch2):
                    region[i,j,k] = target67_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
                    pre.append(target67_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)])
        past = 255/max(pre)
        region = np.array(region)
        region_past = region*past
        complex_test.append(region)
#         region_small = region[int((patch0-pdt_patch0)/2):int((patch0-pdt_patch0)/2)+pdt_patch0,int((patch1-pdt_patch1)/2):int((patch1-pdt_patch1)/2)+pdt_patch1,int((patch2-pdt_patch2)/2):int((patch2-pdt_patch2)/2)+pdt_patch2]
#         label_region = np.zeros((pdt_patch0,pdt_patch1,pdt_patch2))
#         for i in range(pdt_patch0):
#             for j in range(pdt_patch1):
#                 for k in range(pdt_patch2):
#                     if region_small[i,j,k]>region_small[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]*0.7 or region_small[i,j,k]>35:
#                         label_region[i,j,k]=1
#         label_region[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]=1
#         label_test.append(label_region)
#         label_valid.append(region_small)
    
#微血管测试集-S70p
a,b,c = target70_3d_normalization.shape
for n in range(len(positive_test70)):
    x,y,z = positive_test70[n]
    x=int(round((x+0.5)*insert0-1))
    y=int(round((y+0.5)*insert1-1))
    z=int(round((z+0.5)*insert2-1))
    pre = []
    if target70_3d_normalization[x,y,z] > 10 and patch0<x<a-patch0 and patch0<y<b-patch0 and patch0<z<c-patch0:
        region = np.zeros((patch0,patch1,patch2))
        for i in range(patch0):
            for j in range(patch1):
                for k in range(patch2):
                    region[i,j,k] = target70_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
                    pre.append(target70_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)])
        past = 255/max(pre)
        region = np.array(region)
        region_past = region*past
        complex_test.append(region)
#         region_small = region[int((patch0-pdt_patch0)/2):int((patch0-pdt_patch0)/2)+pdt_patch0,int((patch1-pdt_patch1)/2):int((patch1-pdt_patch1)/2)+pdt_patch1,int((patch2-pdt_patch2)/2):int((patch2-pdt_patch2)/2)+pdt_patch2]
#         label_region = np.zeros((pdt_patch0,pdt_patch1,pdt_patch2))
#         for i in range(pdt_patch0):
#             for j in range(pdt_patch1):
#                 for k in range(pdt_patch2):
#                     if region_small[i,j,k]>region_small[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]*0.7 or region_small[i,j,k]>35:
#                         label_region[i,j,k]=1
#         label_region[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]=1
#         label_test.append(label_region)
#         label_valid.append(region_small)
#微血管测试集-S77p
a,b,c = target77_3d_normalization.shape
for n in range(len(positive_test77)):
    x,y,z = positive_test77[n]
    x=int(round((x+0.5)*insert0-1))
    y=int(round((y+0.5)*insert1-1))
    z=int(round((z+0.5)*insert2-1))
    pre = []
    if target77_3d_normalization[x,y,z] > 10 and patch0<x<a-patch0 and patch0<y<b-patch0 and patch0<z<c-patch0:
        region = np.zeros((patch0,patch1,patch2))
        for i in range(patch0):
            for j in range(patch1):
                for k in range(patch2):
                    region[i,j,k] = target77_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
        complex_test.append(region)
        pre.append(target77_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)])
        past = 255/max(pre)
        region = np.array(region)
        region_past = region*past
        complex_test.append(region)
#         region_small = region[int((patch0-pdt_patch0)/2):int((patch0-pdt_patch0)/2)+pdt_patch0,int((patch1-pdt_patch1)/2):int((patch1-pdt_patch1)/2)+pdt_patch1,int((patch2-pdt_patch2)/2):int((patch2-pdt_patch2)/2)+pdt_patch2]
#         label_region = np.zeros((pdt_patch0,pdt_patch1,pdt_patch2))
#         for i in range(pdt_patch0):
#             for j in range(pdt_patch1):
#                 for k in range(pdt_patch2):
#                     if region_small[i,j,k]>region_small[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]*0.7 or region_small[i,j,k]>35:
#                         label_region[i,j,k]=1
#         label_region[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]=1
#         label_test.append(label_region)
#         label_valid.append(region_small)

In [ ]:
y_test_1 = np.ones(len(complex_test))
half_test = len(complex_test)

In [ ]:
#微血管测试集-S57n
a,b,c = target_3d_normalization.shape  
for n in range(len(negative_test)):
    x,y,z = negative_test[n]
#     x=int(round(x*insert0))
#     y=int(round(y*insert1))
#     z=int(round(z*insert2))
    pre = []
    #if target_3d_normalization[x,y,z] < 30:
    region = np.zeros((patch0,patch1,patch2))
    for i in range(patch0):
        for j in range(patch1):
            for k in range(patch2):
                region[i,j,k] = target_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
                pre.append(target_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)])
    past = 255/max(pre)
    region = np.array(region)
    region_past = region*past
    complex_test.append(region)
#     region_small = region[int((patch0-pdt_patch0)/2):int((patch0-pdt_patch0)/2)+pdt_patch0,int((patch1-pdt_patch1)/2):int((patch1-pdt_patch1)/2)+pdt_patch1,int((patch2-pdt_patch2)/2):int((patch2-pdt_patch2)/2)+pdt_patch2]
#     label_region = np.zeros((pdt_patch0,pdt_patch1,pdt_patch2))
#     for i in range(pdt_patch0):
#         for j in range(pdt_patch1):
#             for k in range(pdt_patch2):
#                 if region_small[i,j,k]>region_small[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]*1.2 and region_small[i,j,k]>40:
#                     label_region[i,j,k]=1
#     label_region[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]=0
#     label_test.append(label_region)
#     label_valid.append(region_small)
#S65N
for n in range(len(negative_test65)):
    x,y,z = negative_test65[n]
#     x=int(round(x*insert0))
#     y=int(round(y*insert1))
#     z=int(round(z*insert2))
    pre = []
    #if target65_3d_normalization[x,y,z] < 30:
    region = np.zeros((patch0,patch1,patch2))
    for i in range(patch0):
        for j in range(patch1):
            for k in range(patch2):
                region[i,j,k] = target65_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
                pre.append(target_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)])
    past = 255/max(pre)
    region = np.array(region)
    region_past = region*past
    complex_test.append(region)
#     region_small = region[int((patch0-pdt_patch0)/2):int((patch0-pdt_patch0)/2)+pdt_patch0,int((patch1-pdt_patch1)/2):int((patch1-pdt_patch1)/2)+pdt_patch1,int((patch2-pdt_patch2)/2):int((patch2-pdt_patch2)/2)+pdt_patch2]
#     label_region = np.zeros((pdt_patch0,pdt_patch1,pdt_patch2))
#     for i in range(pdt_patch0):
#         for j in range(pdt_patch1):
#             for k in range(pdt_patch2):
#                 if region_small[i,j,k]>region_small[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]*1.2 and region_small[i,j,k]>40:
#                     label_region[i,j,k]=1
#     label_region[int((pdt_patch0-1)/2),int((pdt_patch1-1)/2),int((pdt_patch2-1)/2)]=0
#     label_test.append(label_region)
#     label_valid.append(region_small)
    
# #微血管测试集-S67n
# a,b,c = target67_3d_normalization.shape
# for n in range(len(negative_test67)):
#     x,y,z = negative_test67[n]
# #     x=int(round(x*insert0))
# #     y=int(round(y*insert1))
# #     z=int(round(z*insert2))
#     if target67_3d_normalization[x,y,z] < 30 and patch0<x<a-patch0 and patch0<y<b-patch0 and patch0<z<c-patch0:
#         region = np.zeros((patch0,patch1,patch2))
#         for i in range(patch0):
#             for j in range(patch1):
#                 for k in range(patch2):
#                     region[i,j,k] = target67_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
#         complex_test.append(region)
    
# #微血管测试集-S70n
# a,b,c = target70_3d_normalization.shape
# for n in range(len(negative_test70)):
#     x,y,z = negative_test70[n]
# #     x=int(round(x*insert0))
# #     y=int(round(y*insert1))
# #     z=int(round(z*insert2))
#     if target70_3d_normalization[x,y,z] < 30 and patch0<x<a-patch0 and patch0<y<b-patch0 and patch0<z<c-patch0:
#         region = np.zeros((patch0,patch1,patch2))
#         for i in range(patch0):
#             for j in range(patch1):
#                 for k in range(patch2):
#                     region[i,j,k] = target70_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
#         complex_test.append(region)
    
# #微血管测试集-S77n
# a,b,c = target77_3d_normalization.shape
# for n in range(len(negative_test77)):
#     x,y,z = negative_test77[n]
#     if target77_3d_normalization[x,y,z] < 30:
#         region = np.zeros((patch0,patch1,patch2))
#         for i in range(patch0):
#             for j in range(patch1):
#                 for k in range(patch2):
#                     region[i,j,k] = target77_3d_normalization[int(x-(patch0-1)/2+i),int(y-(patch1-1)/2+j),int(z-(patch2-1)/2+k)]
#         complex_test.append(region)

In [ ]:
y_test_0 = np.zeros(len(complex_test)-half_test)
y_test_before = np.hstack((y_test_1,y_test_0))

In [ ]:
y_test_0 = np.zeros(1)
y_test_1 = np.ones(1)
y_train_0 = np.zeros(1)
y_train_1 = np.ones(1)
y_test_before = np.hstack((y_test_1,y_test_0))
y_train_before = np.hstack((y_train_1,y_train_0))
complex_train = []
complex_test = []
a = np.zeros((25,25,25))
for n in range(2):
    complex_train.append(a)
    complex_test.append(a)
complex_train = np.array(complex_train)
complex_test = np.array(complex_test)

In [ ]:
#numpy化
X_train_before = np.array(complex_train)
X_test_before = np.array(complex_test)
#张量化
X_train_before = X_train_before.reshape(X_train_before.shape[0], patch2, patch1, patch0, 1).astype('float32')
X_test_before = X_test_before.reshape(X_test_before.shape[0], patch2, patch1, patch0, 1).astype('float32')
#标准化
X_train = X_train_before/1
X_test = X_test_before/1
#标签向量化
y_train = np_utils.to_categorical(y_train_before)
y_test = np_utils.to_categorical(y_test_before)
num_classes = y_test.shape[1]

In [ ]:
from tensorflow.python.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, mode='auto')

In [ ]:
w1 = 2
w2 = 0.2

weight_for_0 = (1 / len(y_test_0))*(len(y_test_0)+len(y_test_1))/w1 
weight_for_1 = (1 / len(y_test_1))*(len(y_test_0)+len(y_test_1))/w2

class_weight1 = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

model2 = Sequential()#是一个简单的线性结构，没有多余分支，是多个网络层的堆叠
#model.add(GaussianNoise(stddev))
model2.add(Conv3D(128, (3, 3, 3), padding='valid',input_shape=(patch2, patch1, patch0, 1), activation='relu'))#128
model2.add(MaxPooling3D(pool_size=(2, 2, 2)))
model2.add(Conv3D(128, (3, 3, 3), padding='valid',activation='relu'))
#model2.add(Conv3D(128, (3, 3, 3), padding='valid',activation='relu'))#128
model2.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model2.add(Conv3D(128, (3, 3, 3), padding='valid',activation='relu'))
model2.add(Conv3D(128, (3, 3, 3), padding='valid',activation='relu'))
#model.add(Dropout(0.2))
model2.add(Flatten())#将输入“压平”，即把多维的输入一维化，常用在从卷积层到全连接层的过渡
model2.add(Dense(128, activation='relu'))#Dense是一个全连接层，默认的激活函数是线性的linear
#model.add(Dense(128, activation='relu'))
model2.add(Dense(16, activation='relu'))
#model.add(Dense(16, activation='relu'))
model2.add(Dense(2, activation='sigmoid'))#又全连接一次
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=25, verbose=2, callbacks=[reduce_lr],class_weight=class_weight1)

#model2.save(r'/home/lzx/csvd_afterinsert_25iso_picture.h5')
model2.save('picture.h5')

In [ ]:
from tensorflow.python.keras.utils.vis_utils import plot_model
plot_model(model2, to_file='model1.png', show_shapes=True)

In [ ]:
file = open('/home/lzx/accuracy.txt','w')
file.write(str(accuracy))
file.close()